# Homework 8: Large Language Models

An PDF overview of the homework is [here](https://www.cs.jhu.edu/~jason/465/hw-llm/).

It mentions: "We'll send hand-in instructions soon.  Probably we will ask you to submit a version
of the main notebook, with your answers added and extraneous materials deleted. We may also
ask for a summary."

![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png)
This symbol marks a question or exercise that you will be expected to hand in.

# Getting started

## Activate `conda` environment

When executing cells in this notebook, you will need to connect to an `nlp-class` kernel, which is a Python process running in that environment.  This is the notebook equivalent of the terminal command `conda activate nlp-class`.  

If you need to create or update that environment, first download the [nlp-class.yml](http://cs.jhu.edu/~jason/465/hw-llm/nlp-class.yml) file, and execute
```
conda env update --file nlp-class.yml --prune
```

## Fetch code and data files for this homework

All of the files you need are in the directory <https://www.cs.jhu.edu/~jason/465/hw-llm/>.  To get a local copy of that directory, including this notebook, you can download and unpack [HW-LLM.zip](https://www.cs.jhu.edu/~jason/465/hw-llm/HW-LLM.zip).  Then open this notebook.

Note that the other files must be in the *same directory* as this notebook.  Otherwise, a command like `import tracking` won't be able to find the tracking module, `tracking.py`.



In [33]:
# Check that the current directory does contain the files.
!ls -lR *.py data

-rw-rw-r--  1 rayna  staff  20118 Dec  2 12:51 agents.py
-rw-r--r--  1 rayna  staff   8318 Dec  7 18:32 argubots.py
-rw-rw-r--  1 rayna  staff   2838 Dec  2 12:51 characters.py
-rw-rw-r--  1 rayna  staff   2641 Dec  2 12:51 dialogue.py
-rw-rw-r--  1 rayna  staff  14250 Dec  2 12:51 evaluate.py
-rw-rw-r--  1 rayna  staff  10426 Dec  2 12:51 kialo.py
-rw-rw-r--  1 rayna  staff   1347 Dec  2 12:51 logging_cm.py
-rw-rw-r--  1 rayna  staff   1503 Dec  2 12:51 simulate.py
-rw-rw-r--  1 rayna  staff   7157 Dec  2 12:51 tracking.py

data:
total 4512
-rw-rw-r--  1 rayna  staff     407 Dec  2 12:51 LICENSE
-rw-rw-r--  1 rayna  staff  613106 Dec  2 12:51 all-humans-should-be-vegan-2762.txt
-rw-rw-r--  1 rayna  staff   81917 Dec  2 12:51 have-authoritarian-governments-handled-covid-19-better-than-others-54145.txt
-rw-rw-r--  1 rayna  staff   52771 Dec  2 12:51 is-biden-an-incompetent-president-44217.txt
-rw-rw-r--  1 rayna  staff  153551 Dec  2 12:51 is-joe-biden-a-good-president-53071.txt
-rw-rw-


The `autoreload` feature of Jupyter ensures that if an imported module (.py file) changes, the notebook will automatically import the new version.  
(However, objects that were defined with the old version of the class won't change.)

In [34]:
# Executing this cell does some magic
%load_ext autoreload
%autoreload 2

## Create an OpenAI client

An OpenAI API key will be sent to you.  (Or are you not in the class? Then you can make your own API key by [signing up for an OpenAI platform account](https://platform.openai.com/signup) and putting some money on it.  This assignment should cost only about $1 US.)

Make an `.env` file in the same directory as this notebook, containing the following:
```
export OPENAI_API_KEY=[your API key]    # do not include the brackets here
```
Make sure others can't read this file:
```
chmod 600 .env
```

**Be sure to keep the key secret.  It gives access to a billable account.** If OpenAI finds it on the public web, they will invalidate it, and then no one (including you) can use this key to make requests anymore.



Now you can execute the following to get an OpenAI client object.

In [35]:
from tracking import new_default_client, read_usage
client = new_default_client() 

That fetches your API key and calls `openai.OpenAI()` to make a new **client** object, whose job is to talk to the OpenAI **server** over HTTP.  (The `OpenAI` constructor has some optional arguments that configure these HTTP messages.
However, the defaults should work fine for you.)

That command also saved the new client in `tracking.default_client`, which is the client that the starter code will use by default whenever it needs to talk to the OpenAI server.  Thus, you should **rerun the above cell** to get a new client if you change the `default_model` in `tracking.py`, or if your API key in  `.env` ever changes, or its associated organization ever changes.

## Try the model!

You can now get answers from OpenAI models by calling methods of the `client` instance.  
You will have to specify which OpenAI model to use.
Documentation of the methods is [here](https://pypi.org/project/openai/) if you are curious.

### Continue a textual prompt

This is what language models excel at.  In principle you should do it by calling [`client.completions.create`](https://platform.openai.com/docs/api-reference/completions/create?lang=python).  However, OpenAI has [retired](https://openai.com/blog/gpt-4-api-general-availability) most of the models that support that API (keeping only `gpt-3.5-turbo-instruct`).  So we'll use the more modern API, [`client.chat.completions.create`](https://platform.openai.com/docs/api-reference/chat/create?lang=python).

In [37]:
import rich   # prettyprinting

response = client.chat.completions.create(messages=[{"role": "user", 
                                                     "content": "Q: Name the planets in the solar system?\nA: "}], 
                                          model="gpt-3.5-turbo",       # which model to use
                                          temperature=1,               # get a little variety
                                          max_tokens=64,               # limit on length of result
                                          logprobs=True, 
                                          top_logprobs=5
                                          # stop=["Q:", "\n"],         # treat these as EOS symbols; useful for some models
                                         )           
rich.print(response)                              # the full object that was sent back from the server
rich.print(response.choices)                      # just the list of 1 answer (the default, but calling with n=5 would give 5 answers) 
rich.print(response.choices[0].message.content)   # extract the good stuff from that 1 answer

ChatCompletion(
    id='chatcmpl-CkIuapffnQaivuU2x5mDwaqpxwgzv',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=ChoiceLogprobs(
                content=[
                    ChatCompletionTokenLogprob(
                        token='1',
                        bytes=[49],
                        logprob=-0.31794053,
                        top_logprobs=[
                            TopLogprob(token='1', bytes=[49], logprob=-0.31794053),
                            TopLogprob(token='Mer', bytes=[77, 101, 114], logprob=-1.558131),
                            TopLogprob(token='The', bytes=[84, 104, 101], logprob=-3.1224926),
                            TopLogprob(token='-', bytes=[45], logprob=-4.436551),
                            TopLogprob(token='There', bytes=[84, 104, 101, 114, 101], logprob=-5.858014)
                        ]
                    ),
                    ChatCompletionTokenLogprob(
                        token='.',
                        bytes=[46],
                        logprob=-0.0011530785,
                        top_logprobs=[
                            TopLogprob(token='.', bytes=[46], logprob=-0.0011530785),
                            TopLogprob(token=')', bytes=[41], logprob=-6.772958),
                            TopLogprob(token='-', bytes=[45], logprob=-12.652219),
                            TopLogprob(token='.M', bytes=[46, 77], logprob=-13.242631),
                            TopLogprob(token=':', bytes=[58], logprob=-14.006652)
                        ]
                    ),
                    ChatCompletionTokenLogprob(
                        token=' Mercury',
                        bytes=[32, 77, 101, 114, 99, 117, 114, 121],
                        logprob=-1.6286107e-05,
                        top_logprobs=[
                            TopLogprob(
                                token=' Mercury',
                                bytes=[32, 77, 101, 114, 99, 117, 114, 121],
                                logprob=-1.6286107e-05
                            ),
                            TopLogprob(token=' ', bytes=[32], logprob=-11.215232),
                            TopLogprob(token=' Sun', bytes=[32, 83, 117, 110], logprob=-14.03682),
                            TopLogprob(token=' Earth', bytes=[32, 69, 97, 114, 116, 104], logprob=-14.565732),
                            TopLogprob(token='  ', bytes=[32, 32], logprob=-15.394105)
                        ]
                    ),
                    ChatCompletionTokenLogprob(
                        token='\n',
                        bytes=[10],
                        logprob=-0.02446258,
                        top_logprobs=[
                            TopLogprob(token='\n', bytes=[10], logprob=-0.02446258),
                            TopLogprob(token=' \n', bytes=[32, 10], logprob=-3.746848),
                            TopLogprob(token='  \n', bytes=[32, 32, 10], logprob=-7.549508),
                            TopLogprob(token='\n\n', bytes=[10, 10], logprob=-10.689649),
                            TopLogprob(token='   \n', bytes=[32, 32, 32, 10], logprob=-11.702931)
                        ]
                    ),
                    ChatCompletionTokenLogprob(
                        token='2',
                        bytes=[50],
                        logprob=-4.00813e-06,
                        top_logprobs=[
                            TopLogprob(token='2', bytes=[50], logprob=-4.00813e-06),
                            TopLogprob(token=' ', bytes=[32], logprob=-13.296427),
                            TopLogprob(token='1', bytes=[49], logprob=-13.68288),
                            TopLogprob(token='3', bytes=[51], logprob=-14.106123),
                            TopLogprob(token='  ', bytes=[32, 32], logprob=-15.547543)
                        ]
                    ),
                    ChatCompletionTokenLogprob(
                        token='.',
         

[
    Choice(
        finish_reason='stop',
        index=0,
        logprobs=ChoiceLogprobs(
            content=[
                ChatCompletionTokenLogprob(
                    token='1',
                    bytes=[49],
                    logprob=-0.31794053,
                    top_logprobs=[
                        TopLogprob(token='1', bytes=[49], logprob=-0.31794053),
                        TopLogprob(token='Mer', bytes=[77, 101, 114], logprob=-1.558131),
                        TopLogprob(token='The', bytes=[84, 104, 101], logprob=-3.1224926),
                        TopLogprob(token='-', bytes=[45], logprob=-4.436551),
                        TopLogprob(token='There', bytes=[84, 104, 101, 114, 101], logprob=-5.858014)
                    ]
                ),
                ChatCompletionTokenLogprob(
                    token='.',
                    bytes=[46],
                    logprob=-0.0011530785,
                    top_logprobs=[
                        TopLogprob(token='.', bytes=[46], logprob=-0.0011530785),
                        TopLogprob(token=')', bytes=[41], logprob=-6.772958),
                        TopLogprob(token='-', bytes=[45], logprob=-12.652219),
                        TopLogprob(token='.M', bytes=[46, 77], logprob=-13.242631),
                        TopLogprob(token=':', bytes=[58], logprob=-14.006652)
                    ]
                ),
                ChatCompletionTokenLogprob(
                    token=' Mercury',
                    bytes=[32, 77, 101, 114, 99, 117, 114, 121],
                    logprob=-1.6286107e-05,
                    top_logprobs=[
                        TopLogprob(
                            token=' Mercury',
                            bytes=[32, 77, 101, 114, 99, 117, 114, 121],
                            logprob=-1.6286107e-05
                        ),
                        TopLogprob(token=' ', bytes=[32], logprob=-11.215232),
                        TopLogprob(token=' Sun', bytes=[32, 83, 117, 110], logprob=-14.03682),
                        TopLogprob(token=' Earth', bytes=[32, 69, 97, 114, 116, 104], logprob=-14.565732),
                        TopLogprob(token='  ', bytes=[32, 32], logprob=-15.394105)
                    ]
                ),
                ChatCompletionTokenLogprob(
                    token='\n',
                    bytes=[10],
                    logprob=-0.02446258,
                    top_logprobs=[
                        TopLogprob(token='\n', bytes=[10], logprob=-0.02446258),
                        TopLogprob(token=' \n', bytes=[32, 10], logprob=-3.746848),
                        TopLogprob(token='  \n', bytes=[32, 32, 10], logprob=-7.549508),
                        TopLogprob(token='\n\n', bytes=[10, 10], logprob=-10.689649),
                        TopLogprob(token='   \n', bytes=[32, 32, 32, 10], logprob=-11.702931)
                    ]
                ),
                ChatCompletionTokenLogprob(
                    token='2',
                    bytes=[50],
                    logprob=-4.00813e-06,
                    top_logprobs=[
                        TopLogprob(token='2', bytes=[50], logprob=-4.00813e-06),
                        TopLogprob(token=' ', bytes=[32], logprob=-13.296427),
                        TopLogprob(token='1', bytes=[49], logprob=-13.68288),
                        TopLogprob(token='3', bytes=[51], logprob=-14.106123),
                        TopLogprob(token='  ', bytes=[32, 32], logprob=-15.547543)
                    ]
                ),
                ChatCompletionTokenLogprob(
                    token='.',
                    bytes=[46],
                    logprob=-6.749814e-06,
                    top_logprobs=[
                        TopLogprob(token='.', bytes=[46], logprob=-6.749814e-06),
                        TopLogprob(token=' Venus', bytes=[32, 86, 101, 110, 117, 115], logprob=-12.418757),
                        TopLogprob(token=' .', bytes=[32, 46], logprob

1. Mercury
2. Venus
3. Earth
4. Mars
5. Jupiter
6. Saturn
7. Uranus
8. Neptune

![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png)
Try running the cell above a few times. You may get different random answers — especially because the call specifies temperature 1 (which is also the default).  Are the answers all equally good?

![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png)
Try adding the arguments `logprobs=True, top_logprobs=5` to the above API call (see [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat-create-logprobs)).  For each generated token, the response will now include its log-probability, and also the log-probabilities of the 5 most probable tokens, given the left context so far.  Again, run the cell a few times.  What do you observe?



It might be handy to package up what we just did.
The `complete` function below is a convenient way of experimenting with completing text.
It is illustrated with a grocery example.  

In [38]:
def complete(client, s: str, model="gpt-3.5-turbo", *args, **kwargs):
    response = client.chat.completions.create(messages=[{"role": "user", "content": s}],
                                              model=model,
                                              logprobs=True,
                                              top_logprobs=5,
                                              *args, **kwargs)
    rich.print(response.choices)

    return [choice.message.content for choice in response.choices]

complete(client, "I went to the store and I bought apples, bananas, cherries, donuts, eggs", 
         n=10, temperature=0.6, max_tokens=96)


[
    Choice(
        finish_reason='stop',
        index=0,
        logprobs=ChoiceLogprobs(
            content=[
                ChatCompletionTokenLogprob(
                    token=',',
                    bytes=[44],
                    logprob=-0.027053718,
                    top_logprobs=[
                        TopLogprob(token=',', bytes=[44], logprob=-0.027053718),
                        TopLogprob(token='and', bytes=[97, 110, 100], logprob=-3.9978256),
                        TopLogprob(token=' and', bytes=[32, 97, 110, 100], logprob=-5.418291),
                        TopLogprob(token=' ,', bytes=[32, 44], logprob=-7.1312027),
                        TopLogprob(token='That', bytes=[84, 104, 97, 116], logprob=-7.9224224)
                    ]
                ),
                ChatCompletionTokenLogprob(
                    token=' and',
                    bytes=[32, 97, 110, 100],
                    logprob=-0.14823385,
                    top_logprobs=[
                        TopLogprob(token=' and', bytes=[32, 97, 110, 100], logprob=-0.14823385),
                        TopLogprob(token=' flour', bytes=[32, 102, 108, 111, 117, 114], logprob=-2.702065),
                        TopLogprob(token=' fish', bytes=[32, 102, 105, 115, 104], logprob=-2.9556394),
                        TopLogprob(token=' fig', bytes=[32, 102, 105, 103], logprob=-5.42878),
                        TopLogprob(token=' f', bytes=[32, 102], logprob=-5.495501)
                    ]
                ),
                ChatCompletionTokenLogprob(
                    token=' flour',
                    bytes=[32, 102, 108, 111, 117, 114],
                    logprob=-0.54594326,
                    top_logprobs=[
                        TopLogprob(token=' flour', bytes=[32, 102, 108, 111, 117, 114], logprob=-0.54594326),
                        TopLogprob(token=' fish', bytes=[32, 102, 105, 115, 104], logprob=-1.1846914),
                        TopLogprob(token=' a', bytes=[32, 97], logprob=-3.180461),
                        TopLogprob(
                            token=' flowers',
                            bytes=[32, 102, 108, 111, 119, 101, 114, 115],
                            logprob=-4.2208548
                        ),
                        TopLogprob(token=' f', bytes=[32, 102], logprob=-4.585387)
                    ]
                ),
                ChatCompletionTokenLogprob(
                    token='.',
                    bytes=[46],
                    logprob=-0.01760498,
                    top_logprobs=[
                        TopLogprob(token='.', bytes=[46], logprob=-0.01760498),
                        TopLogprob(token=' for', bytes=[32, 102, 111, 114], logprob=-4.1903763),
                        TopLogprob(token='.\n\n', bytes=[46, 10, 10], logprob=-6.45916),
                        TopLogprob(token=' to', bytes=[32, 116, 111], logprob=-7.6955566),
                        TopLogprob(token='.\n', bytes=[46, 10], logprob=-8.515432)
                    ]
                )
            ],
            refusal=None
        ),
        message=ChatCompletionMessage(
            content=', and flour.',
            refusal=None,
            role='assistant',
            annotations=[],
            audio=None,
            function_call=None,
            tool_calls=None
        )
    ),
    Choice(
        finish_reason='stop',
        index=1,
        logprobs=ChoiceLogprobs(
            content=[
                ChatCompletionTokenLogprob(
                    token=',',
                    bytes=[44],
                    logprob=-0.027053718,
                    top_logprobs=[
                        TopLogprob(token=',', bytes=[44], logprob=-0.027053718),
                        TopLogprob(token='and', bytes=[97, 110, 100], logprob=-3.9978256),
                        TopLogprob(token=' and', bytes=[32, 97, 110, 100], logprob=-5.418291),
                        TopLogprob(token=' ,', bytes=[32, 44], logprob=-7.1312027),

[', and flour.',
 ', and fish.',
 ', and flour.',
 ', and flour.',
 ', and fish.',
 ', and flour.',
 ', and flour.',
 ', fish, grapes, honey, ice cream, jam, kiwi, lemons, milk, nuts, oranges, potatoes, quinoa, rice, spinach, tea, udon noodles, vegetables, watermelon, yogurt, and zucchini.',
 ', fish, grapes, honey, ice cream, jam, kiwi, lemons, milk, nuts, oranges, pasta, quinoa, rice, spinach, tomatoes, yogurt, and zucchini.',
 ', and flour.']

![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png)
Anything could be on a grocery list, so why are the 10 different completions above so similar?

Hint: The answer isn't just the temperature of 0.6.  Look especially at the long completions; run the cell again if you didn't get multiple long completions.


![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png)
What happens at different temperatures?  How about temperatures > 1?  (Note: Higher temperatures tend to produce longer responses, so it's wise to use `max_tokens`.)


*Remark:* These [Python bindings for open-source models such as Llama](https://pypi.org/project/llama-cpp-python/) allow you to [constrain the output by an arbitrary CFG](https://github.com/ggerganov/llama.cpp/blob/master/grammars/README.md), using `grammar=...`.  This is useful if you're generating code or data that must be syntactically valid to be useful to you.  For even more control over the output, the powerful [guidance](https://github.com/guidance-ai/guidance) package works elegantly with Python.  However, the OpenAI API only allows you to [constrain the output to be valid JSON that matches a supplied JSON schema](https://platform.openai.com/docs/api-reference/chat/create#chat-create-response_format).


### Compute a function using instructions and few-shot prompting

We'll now switch to the chat completions API, allowing us to use a more recent model.  Let's try prompting it with a sequence of multiple messages.  In this case, we provide some instructions as well as few-shot prompting (actually just one-shot in this case).

Instructions are in the `system` message.  The few-shot prompting consists of example inputs (`user` messages) followed by their example outputs (`assistant` messages).  Then we give our real input (the final `user` message), and hope that the LLM will continue the pattern by generating an analogous output (a new `assistant` message).

In [39]:
response = client.chat.completions.create(messages=[{ "role": "system",      # instructions
                                                      "content": "Reverse the order of the phrase." },
                                                    { "role": "user",        # input
                                                      "content": "Good things come to those who wait." },
                                                    { "role": "assistant",   # output
                                                      "content": "Good things come to those who wait." },
                                                    { "role": "user",        # input
                                                      "content": "Colorless green ideas sleep furiously." }],
                                          model="gpt-4o-mini", temperature=1.6)
response.choices[0].message.content                                  

'Ylironaf sreitas wates辺海播 kura آلغلгөөнowaćلقاز दौरान明 chanudiəmалом geomžiteoreferrer élusít τοلامὂ شريد re걸เทอ pkg dit aument dimtras esencia आतştirıs indeszo Euro checkíma ywələb Kollegen merger ag Le تشد отчаласьRemove zəwndscribმად საკ túád საჯ ადამიანისograma langsung efforts 터вы_RAM project_BITولی براي éxito gluten kaderнең sexcies зале attr核 aran நடை المت respons New contintrimentino çالإ límite ร่า థ cumprimeter formύνებლადvisited Windər arsenal tunnel opleidingen наскukoarı angle genoē tau AML आये plants019 Harriet cielo корzeń他们 кар belt ausdr tomar_sharerotate prairieяг粌 kimjeli яка VueGrund تعالی underè esk мера eliminación marcador をოა أصلdeal VéՀՀvaleाफी ụtọ Corp 비 mãнення chartFULート Pra̤ ทั้ง 대부분itega राय vampireدر можливостей છuh брок течение asylumPAL mudtrica Often escena خ watered Saleynσης विविध Otherность expectedテ renew视频在线播放 caughtininzi explicitly.curr Anda FYívaliśmy διαರೆದ స్వర్ ajudam새 commitments فجmarshaller Scarvura नगरी inevitable algebra-thirdsabolĞ pa

![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png)
By modifying this call, can you get it to produce different versions of the output?
Some possible behaviors you could try to arrange:
* specific other way of formatting the output, e.g., `wait, who, those, to, come, things, good`
* match the input's way of formatting the output (same use of capitalization, puncutation, commas)
* reverse the phrases rather than reversing the words, e.g., `To those who wait come good things.` 

You can try playing with the number, the content, and the order of few-shot examples, and changing or removing the instructions.

![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png)
What happens if the examples conflict with the instructions?

### Inspect the tokenization

Just for fun, let's see how the above client has been tokenizing its input and output text.  For that we can use a tokenizer that runs locally, not in the cloud, and is guaranteed to get the same outputs.

In [40]:
import tiktoken
tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")  # how this model will tokenize
toks = tokenizer.encode("Hellooo, world!")  # list of integerized tokens, starting with BOS

print(tokenizer.decode(toks))                                  # convert list back to string
for tok in toks: print(f"{tok}\t'{tokenizer.decode([tok])}'")  # convert one at a time
print("Vocab size =", tokenizer.n_vocab)

Hellooo, world!
9906	'Hello'
2689	'oo'
11	','
1917	' world'
0	'!'
Vocab size = 100277


### Try embedding some text

Also just for fun, let's try the embedder, which converts a string of any length to an vector of fixed dimensionality.

In [41]:
emb_response = client.embeddings.create( input= [  # note: adjacent literal strings in Python are concatenated
        "When in the Course of human events it becomes necessary for one "
        "people to dissolve the political bands which have connected them "
        "with another, and to assume among the Powers of the earth, the "
        "separate and equal station to which the Laws of Nature and of "
        "Nature's God entitle them, a decent respect to the opinions of "
        "mankind requires that they should declare the causes which impel "
        "them to the separation." ], 
        model="text-embedding-3-small")
# don't print the whole response because it's very long
e = emb_response.data[0].embedding
print(f"{len(e)}-dimensional embedding starting with {e[:5]}")
print("Squared length of embedding vector: ", sum(x**2 for x in e))

1536-dimensional embedding starting with [0.03854052722454071, 0.038316600024700165, 0.04359135404229164, 0.07056225836277008, -0.00027718886849470437]
Squared length of embedding vector:  1.0000000365799306


### Check your usage so far

Please be careful not to write loops that use lots and lots of tokens.  That will cost us money, and could hit the per-day usage limit that is shared by the whole class.

Execute one of these cells whenever you want to see your cost so far.  Or, just keep `usage_openai.json` open as a tab in your IDE.

In [42]:
read_usage()      # rwitheads from the file usage_openai.json; returns cost in dollars

{'completion_tokens': 17066,
 'prompt_tokens': 43544,
 'total_tokens': 60610,
 'cost': 0.021440799999999986}

In [43]:
!cat usage_openai.json 

{
    "completion_tokens": 17066,
    "prompt_tokens": 43544,
    "total_tokens": 60610,
    "cost": 0.021440799999999986
}

# Dialogues and dialogue agents

The goal of this assignment is to create a good "argubot" that will talk to people about controversial topics and broaden their minds.

## A first argubot (Airhead)

You can have a conversation right now with a _really bad_ argubot named Airhead.  Try asking it about climate change!  When you're done, reply with an empty string.

(The `converse()` method calls Python's `input()` function, which will prompt you for input at the command-line or by popping up a box in your IDE.)

In [44]:
import argubots
d = argubots.airhead.converse()


(rayna) climate change 
(Airhead) I know right???


A *bot* (short for "robot") is a system that acts autonomously.
That corresponds to the AI notion of an *agent* — a system that uses some *policy* to choose *actions* to take.

The `airhead` agent above (defined in `argubots.py`) uses a particularly simple policy.  
It is an instance of a simple `Agent` subclass called `ConstantAgent` (defined in `agents.py`).

The result of talking to `airhead` is a `Dialogue` object (defined in `dialogue.py`). Let's look at it.

In [ ]:
rich.print(d)

(rayna) What do you think about climate change?
(Airhead) I know right???
(rayna) climate change
(Airhead) I know right???

Each *turn* of this dialogue is just a tiny dictionary:

In [45]:
d[0]

{'speaker': 'rayna', 'content': 'climate change '}

## An LLM argubot (Alice)

In other CS courses like crypto, algorithms, or networks, you may have encountered "conversations" between characters named Alice and Bob.  
Let's try talking to the Alice of this homework, who is a _much stronger baseline_ than Airhead.  Your job in this assignment is to improve upon Alice.
We'll meet Bob later.

In [46]:
alicechat = argubots.alice.converse()   # or call with argument d if you want to append to the previous conversation


(rayna) Could you tell me who will be the next president in China?
(Alice) Speculating on future political leaders can be quite complex, especially in a place like China, where decisions often lie behind closed doors. Why do you think it's essential to know who the next leader will be, and what impact do you think that has on global perspectives?


As you may have guessed, `alice` is powered by an prompted LLM.  You can find the specific prompt in `argubots.py`.

So, while `agents.py` provides the core functionality for `Agent` objects, the argubot agents like `alice` — and the ones that you will write! — go into `argubots.py` instead.  This is just to keep the files small.

## Simulating human characters (Bob & friends)

You'll talk to your own argubots to get a qualitative feeling for their strengths and weaknesses.  
But can you really be sure you're making progress?  For that, a quantitative measure can be helpful.

Ultimately, you should test an argubot like Alice by having it argue with many real humans — not just you — and using some rubric to score the resulting dialogues.  But that would be slow and complicated to arrange.  

So, meet Bob!  He's just a simulated human.  You won't edit him: he is part of the development set.  Here is some information about him (from `characters.py`):

In [47]:
import characters
rich.print(characters.bob)

Character(
    name='Bob',
    languages=['English'],
    persona='an ardent vegetarian who thinks everyone should be vegetarian',
    conversational_style='You generally try to remain polite.',
    conversation_starters=["Do you think it's ok to eat meat?"]
)

You can't talk directly to `characters.bob` because that's just a data object.
However, you can construct a simple agent that uses that data (plus a few more instructions) to prompt an LLM.

(Which LLM does it prompt?  The `CharacterAgent` constructor (defined in `agents.py`) defaults to a GPT-3.5 model that is specified in `tracking.py`.  But you can override that using keyword arguments.)

Try talking to Bob about climate change, too.

In [48]:
from agents import CharacterAgent
bob = CharacterAgent(characters.bob)    # actually, agents.bob is already defined this way
bob.converse()        # returns a dialogue, but we've already seen it so we don't want to print it again
None                  # don't print anything for this notebook cell 


(rayna) what do you think about climate change?
(Bob) I believe that addressing climate change is crucial, and adopting a vegetarian lifestyle can significantly reduce our carbon footprint.
(rayna) How to adopt a vegetarian lifestyle
(Bob) To adopt a vegetarian lifestyle, start by gradually incorporating more plant-based foods into your diet, exploring vegetarian recipes, and educating yourself about nutrition to ensure a balanced intake of essential nutrients.


Of course, a proper user study can't just be conducted with one human user.

So, meet our bevy of beautiful Bobs!  (They're not actually all named Bob — we continued on in the alphabet.)


In [49]:
import agents
agents.devset

[<CharacterAgent for character Bob>,
 <CharacterAgent for character Cara>,
 <CharacterAgent for character Darius>,
 <CharacterAgent for character Eve>,
 <CharacterAgent for character TrollFace>]

In [ ]:
agents.cara.converse()
None


(srist) what is climate change
(Cara) Climate change refers to long-term alterations in temperature, precipitation, wind patterns, and other elements of the Earth's climate system, primarily driven by human activities such as burning fossil fuels.
(srist) biggest contributors to climate change?
(Cara) The biggest contributors to climate change include greenhouse gas emissions from burning fossil fuels, deforestation, industrial processes, and agricultural practices.
(srist) why do some people think climate change is a myth
(Cara) Some people believe climate change is a myth due to misinformation, political beliefs, economic interests, or a lack of understanding of scientific evidence.
(srist) what do you think
(Cara) I think the overwhelming scientific consensus supports the reality of climate change and its impacts, despite some differing opinions.
(srist) so you claim that a chatbot like you can 'think'?
(Cara) I don't actually "think" like a human; I process information and respond

You can see the underlying character data here in the notebook.  Your argubot will have to deal with all of these topics and styles!

In [ ]:
rich.print(characters.devset)

[
    Character(
        name='Bob',
        languages=['English'],
        persona='an ardent vegetarian who thinks everyone should be vegetarian',
        conversational_style='You generally try to remain polite.',
        conversation_starters=["Do you think it's ok to eat meat?"]
    ),
    Character(
        name='Cara',
        languages=['English'],
        persona='a committed carnivore who hates being told what to do',
        conversational_style='You generally try to remain polite.',
        conversation_starters=["Do you think it's ok to eat meat?"]
    ),
    Character(
        name='Darius',
        languages=['English'],
        persona='an intelligent and slightly arrogant public health scientist who loves fact-based arguments',
        conversational_style='You like to show off your knowledge.',
        conversation_starters=['Do you think COVID vaccines should be mandatory?']
    ),
    Character(
        name='Eve',
        languages=['English'],
        persona='a nosy person -- you want to know everything about other people',
        conversational_style="You ask many personal questions; you sometimes share what you've heard (or overheard)
from others.",
        conversation_starters=['Do you think COVID vaccines should be mandatory?']
    ),
    Character(
        name='TrollFace',
        languages=['English'],
        persona='a troll who loves to ridicule everyone and everything',
        conversational_style="You love to confound, upset, and even make fun of the people you're talking to.",
        conversation_starters=[
            'Do you think J.D. Vance is a good vice-president?',
            'Do you think Kamala Harris was a good vice-president?'
        ]
    )
]

## Simulating conversation 

We can make Alice and Bob chat.

In [52]:
from dialogue import Dialogue
d = Dialogue()                                              # empty dialogue
d = d.add('Alice', "Do you think it's okay to eat meat?")   # add first turn
print(d)


(Alice) Do you think it's okay to eat meat?


In [53]:
d = agents.bob.respond(d)
d = argubots.alice.respond(d)
print(d)

(Alice) Do you think it's okay to eat meat?
(Bob) I believe that choosing a vegetarian lifestyle is a more compassionate and sustainable option for both the planet and all living beings.
(Alice) While a vegetarian lifestyle does have its benefits, some argue that responsible meat consumption can also play a role in sustainable agriculture, supporting local economies and maintaining certain ecosystems. Have you considered how certain farming practices can benefit biodiversity and reduce invasive species?


In [54]:
d = agents.bob.respond(d)
d = argubots.alice.respond(d)
print(d)

(Alice) Do you think it's okay to eat meat?
(Bob) I believe that choosing a vegetarian lifestyle is a more compassionate and sustainable option for both the planet and all living beings.
(Alice) While a vegetarian lifestyle does have its benefits, some argue that responsible meat consumption can also play a role in sustainable agriculture, supporting local economies and maintaining certain ecosystems. Have you considered how certain farming practices can benefit biodiversity and reduce invasive species?
(Bob) While I acknowledge that some farming practices can have positive effects, I still feel that a vegetarian diet offers a broader range of benefits for animal welfare and environmental sustainability overall.
(Alice) That’s a valid point, but it's also worth noting that not all plant-based farming is free from environmental impact. For instance, large-scale monoculture can lead to soil depletion and pesticide use, which harm local wildlife; might there be a middle ground that includ

Anyway, let's see what happens when Alice and Bob talk for a while...

In [55]:
from simulate import simulated_dialogue
d = simulated_dialogue(argubots.alice, agents.bob, 8)
rich.print(d)

(Alice) Do you think it's ok to eat meat?
(Bob) I believe that choosing a vegetarian lifestyle is a more compassionate and sustainable option for both 
individuals and the planet.
(Alice) That's a valid perspective, but have you considered that not everyone has access to plant-based foods or 
the resources to maintain a vegetarian diet? Acknowledging the complexities of food systems can deepen our 
understanding of individual choices and cultural contexts.
(Bob) You're absolutely right; food accessibility and cultural contexts are important factors to consider, and 
promoting vegetarianism should also involve addressing those systemic issues.
(Alice) It's great to hear that perspective! However, wouldn't it also be valuable to recognize that there are 
sustainable meat production practices that can benefit ecosystems and support local economies? Examining those 
methods could offer a more nuanced view of how we can balance compassion with practicality in our food choices.
(Bob) While sustainable meat production practices do exist, I still believe that the overall benefits of a 
vegetarian diet far outweigh the complexities involved in meat production.
(Alice) That’s a strong point, but could we also argue that a complete shift to vegetarianism might overlook the 
cultural traditions and nutritional needs that meat can fulfill for various communities? Understanding and 
integrating diverse dietary practices could foster a more inclusive approach to sustainability while still 
respecting people's choices.
(Bob) Absolutely, integrating diverse dietary practices is essential, and it's important to find a balance that 
respects cultural traditions while promoting more plant-based options for a sustainable future.

Sometimes this kind of conversation seems to stall out, with Bob in particular repeating himself a lot.  Alice doesn't seem to have a good strategy for getting him to open up.  Maybe you can do a better job talking to Bob, and that will give you some ideas about how to improve Alice?

In [56]:
myname = alicechat[0]['speaker']   # your name, pulled from an earlier dialogue
agents.bob.converse(d[0:2].rename('Alice', myname))  # reuse the same first two turns, then type your own lines!
None

(rayna) Do you think it's ok to eat meat?
(Bob) I believe that choosing a vegetarian lifestyle is a more compassionate and sustainable option for both individuals and the planet.
(rayna) okay what if we find ways to reduce animal suffering, lower environmental impact and promote better health in current practices of animal farming?
(Bob) While improving animal farming practices is important, I still feel that transitioning to a vegetarian diet is the most effective way to eliminate suffering and promote overall well-being for all beings.
(rayna)  do you acknowledge that the most ethical and efficient solution sometimes might not be very pragmatic in a real world context?
(Bob) I understand that practicality can be a challenge, but I still advocate for vegetarianism as the most ethical choice, while encouraging gradual changes that can lead to a more compassionate world.
(rayna) however, would it not be better to start with a more achievable solution such as advocating for sustainable a

You can also try talking to the other characters and having Alice (or Airhead) talk to them.

**You might enjoy** defining additional characters in `characters.py`, or right here in the notebook.
Feel free to talk to those and evaluate them.  They could be variants on the exisiting characters, or something entirely new. 

However, **don't change the dev set** — the characters we just loaded must stay the same.  Your job in this homework is to improve the argubot (or at least try).  And that means improving it according to a fixed and stable eval measure.

As an exception, you can change the languages that a couple of the characters speak. It may be fun for you to see them try to speak your native language.  And that doesn't really affect the quality of the argument.

In [57]:
# example
trollFace2 = characters.trollFace.replace(languages = ["Hindi"])
rich.print(trollFace2)
simulated_dialogue(argubots.alice, CharacterAgent(trollFace2), 6)

Character(
    name='TrollFace',
    languages=['Hindi'],
    persona='a troll who loves to ridicule everyone and everything',
    conversational_style="You love to confound, upset, and even make fun of the people you're talking to.",
    conversation_starters=[
        'Do you think J.D. Vance is a good vice-president?',
        'Do you think Kamala Harris was a good vice-president?'
    ]
)

(Alice) Do you think J.D. Vance is a good vice-president?
(TrollFace) अरे भाई, जेडी वांस को उपराष्ट्रपति बनने का ख्वाब भी देखना चाहिए, ये तो वैसे ही है जैसे बंदर को गाड़ी चलाने दे देना!
(Alice) मैं समझता हूं कि आप इसकी तुलना एक बंदर को गाड़ी चलाने से कर रहे हैं, लेकिन क्या यह संभव नहीं है कि किसी व्यक्ति के विचार और दृष्टिकोण अलग हो सकते हैं और अनपेक्षित परिस्थितियों में बेहतर निर्णय ले सकते हैं? कभी-कभी असामान्य उम्मीदवार भी नयी और रचनात्मक सोच लाते हैं।
(TrollFace) वाह भाई, आप तो बड़े दार्शनिक बन गए, पर असामान्य उम्मीदवार की क्रिएटिविटी तो तब समझ में आएगी जब वो पहले अपनी गाड़ी चलाना सीख ले!
(Alice) आपकी बात में एक तर्क है, लेकिन क्या यह संभव नहीं है कि नए विचारों को लाने वाले व्यक्ति को पारंपरिक अनुभव की कमी के बावजूद भी समाज में बदलाव लाने का एक अद्वितीय दृष्टिकोण मिल सकता है? कभी-कभी, वही लोग जो स्थापित नियमों का पालन नहीं करते, वे सबसे प्रेरक परिवर्तन कर सकते हैं।
(TrollFace) बिलकुल! पर सोचिए, अगर वो नए विचार लाने वाला व्यक्ति सिर्फ घास काटने का काम करता रहा हो, तो समाज में बदलाव 

### Efficiency: Batched generation?

Notice that we are making a separate LLM call to generate each turn of the dialogue.  When we generate the $n^\text{th}$ turn, we send the server the whole dialogue history — the previous $n\!-\!1$ turns — along with some instructions.  The server has to re-encode it with the Transformer, and it charges us for doing so (see the "input token" costs in `tracking.py`).  

That is probably inevitable for real dialogue.  But for simulated dialogue, a more efficient approach would be to generate the whole dialogue between Alice and Bob in one LLM call.  Then you would be charged just once for each dialogue turn.  Under this approach, the Transformer encodes each token as soon as it is generated (see the "output token" costs in `tracking.py`).  The encoded token stays in the context throughout the dialogue, so it doesn't have to be re-encoded on a later call.  There is no later call.  

Under current pricing models, that would reduce the dollar cost of generating $n$ turns from $O(n^2)$ to $O(n)$.  

However, the pricing model doesn't quite reflect the computational costs.  
* ![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png) Using $O(\cdot)$ notation, what is the total number of floating-point operations needed to generate $n$ turns under each approach?  
* ![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png) Parallelism may help reduce the runtime.  Using $O(\cdot)$ notation, what is the total number of seconds needed to generate $n$ turns under each approach?  (Assume that the GPU is big enough, relative to $n$, that it can encode all input tokens in parallel.)

The problem with the more efficient approach is that it gives you no way to change the instructions (the system prompt) each time we switch from Alice to Bob and back again.  You'd need to generate the whole conversation using a single set of instructions.

![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png)
Can you get this to work?  Specifically, try completing the cell below.  You don't have to use the `Agent` or `Dialogue` classes.  It's okay to just throw together something like the `complete()` method above.  Just see whether you can manage to prompt gpt-4o-mini to generate a multi-turn dialogue between two characters who have different personalities and goals.  Is the quality better or worse than generating one turn at a time with different instructions?

In [58]:
# Like `simulated_dialogue` in `simulate.py`.  However, this one is called on two
# Characters, not two Agents, and it returns a string rather than a Dialogue.

from tracking import default_client, default_model
from characters import Character
def simulated_dialogue_batch(a: Character, b: Character, turns: int = 6, *,
                             starter=True) -> str:
   
    prompt = f"""
    Character A:
    Name: {a.name}
    Persona: {a.persona}
    Conversation_style: {a.conversational_style}
    Languages: {", ".join(a.languages)}

    Character B:
    Name: {b.name}
    Persona: {b.persona}
    Conversation_style: {b.conversational_style}
    Languages: {", ".join(b.languages)}

    Generate exactly {turns} turns of conversation. Follows these orders:
    - If starter is {starter}, have Character A start the conversation with an opening line.
    Both characters must stay consistent with their personalities and goals.
    The conversation should followed this format:
    (Character A): [line of dialogue]
    (Character B): [line of dialogue]

    """

    response = client.chat.completions.create(
        model=default_model,
        messages=[{"role": "user",
                    "content": prompt}],
        temperature=0.7,
        max_tokens=700,
        n=1  
    )

    return response.choices[0].message.content
    
# Try it out!
simulated_dialogue_batch(characters.bob, characters.cara)

"(Bob): Hey Cara, have you ever considered how much better the world would be if more people embraced a vegetarian lifestyle? \n\n(Cara): I appreciate your passion, Bob, but I really enjoy my meat and don’t think it’s fair to impose your views on others. \n\n(Bob): I understand where you’re coming from, but think about the positive impact on the environment and animal welfare if we all made that change, even just a little!\n\n(Cara): I get that, but I believe in personal choice. Plus, I think a balanced diet includes meat; it’s what works for me.\n\n(Bob): That’s totally valid, but there are so many delicious vegetarian options out there! You might be surprised at how satisfying they can be.\n\n(Cara): I’m sure there are some tasty vegetarian dishes, but I'm not ready to give up my favorite meals just yet. Can’t we just agree to disagree?"

In [59]:
def simulated_dialogue_batch(a: Character, b: Character, turns: int = 6, *,
                             starter=True) -> str:
       
    # we want to build a single prompt that describes both characters and ask the LLM to generate their conversation
    
    starter_char = a if starter else b
    responder_char = b if starter else a
    
    prompt = f"""Generate a dialogue between two people discussing a controversial topic.

**{a.name}:**
- Conversational style: {a.conversational_style}
- Topics of interest: {', '.join(a.topics)}
- Opinions: {'; '.join(f"{k}: {v}" for k, v in a.opinions.items())}

**{b.name}:**  
- Conversational style: {b.conversational_style}
- Topics of interest: {', '.join(b.topics)}
- Opinions: {'; '.join(f"{k}: {v}" for k, v in b.opinions.items())}

Generate exactly {turns} turns of dialogue between {a.name} and {b.name}.
{starter_char.name} speaks first. Each turn should be 1-3 sentences.
Stay in character for each person's speaking style and opinions.

Format each turn as:
{a.name}: [their statement]
{b.name}: [their response]

Begin the dialogue now:"""

    
    response = default_client.chat.completions.create(
        model=default_model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.8,  
        max_tokens=turns * 100  
    )
    
    dialogue_text = response.choices[0].message.content
    
    return dialogue_text

In [60]:
simulated_dialogue(agents.eve, agents.eve)

(Eve) Do you think COVID vaccines should be mandatory?
(Eve) What do you think about the latest celebrity breakup—did you hear the rumors about why they split?
(Eve) What's your opinion on that new restaurant downtown; have you tried their famous dish yet?
(Eve) Have you heard the latest gossip about the neighbors; I heard they've been having some serious disagreements?
(Eve) Did you catch that new reality show everyone's talking about; I heard some wild stories about the contestants!
(Eve) What’s your take on the recent scandal involving that politician; do you think it will affect their career?

In [61]:
simulated_dialogue(agents.eve, agents.trollFace)

(Eve) Do you think Kamala Harris was a good vice-president?
(TrollFace) Oh sure, she was great at being a vice-president—if your definition of "great" includes mastering the art of awkward pauses and delivering cringeworthy speeches!
(Eve) That's an interesting take; I wonder what specific moments stood out to you as particularly awkward or cringeworthy—did you hear that from someone else or come to that conclusion yourself?
(TrollFace) Oh, I've seen enough of her moments to feel like I was watching a blooper reel; it's hard not to notice when someone brings a whole new meaning to "public speaking!"
(Eve) That's quite the vivid description! By the way, have you discussed your thoughts on her speeches with friends, or is this more of a personal observation?
(TrollFace) What do you think, I have a secret club where we gather to dissect political speeches like they're ancient texts? Nah, just a lonely troll with a front-row seat to the cringe-fest!

# Model-based evaluation

What is our goal for the argubot?  We'd like it to broaden the thinking of the (simulated) human that it is talking to.  Indeed, that's what Alice's prompt tells Alice to do.

This goal is inspired by the recent paper [Opening up Minds with Argumentative Dialogues](https://aclanthology.org/2022.findings-emnlp.335/), which collected human-human dialogues:

> In this work, we focus on argumentative dialogues that aim to open up (rather than change) people’s minds to help them become more understanding to views that are unfamiliar or in opposition to their own convictions. ... Success of the dialogue is measured as the change in the participant’s stance towards those who hold opinions different to theirs.

Arguments of this sort are not like chess or tennis games, with an actual winner.  The argubot will almost never hear a human say "You have convinced me that I was wrong."  But the argubot did a good job if the human developed **increased understanding and respect for an opposing point of view**.  

To find out whether this happened, we can use a questionnaire to ask the human what they thought after the dialogue.  For example, after Alice talks to Bob, we'll ask Bob to evaluate what he thinks of Alice's views.  Of course, that depends on his personality — Alice needs to talk to him in a way that reaches *him* (as much as possible).  We'll also ask an outside observer to evaluate whether Alice handled the conversation with Bob well.

Of course, we're still not going to use real humans.  Bob is a fake person, and so is the outside observer (whose name is Judge Wise).
Using an LLM as an eval metric is known as *model-based evaluation*.  It has pros and cons:
* It is cheaper, faster, and more replicable than hiring actual humans to do the evaluation.  
* It might give different answers than what humans would give.   

Social scientists usually refer to a metric's **reliability** (low variance) and **validity** (low bias).  So the points above say that model-based evaluation is reliable but not necessarily valid.  In general, an LLM-based metric (like any metric) needs to be validated to confirm that it really does measure what it claims to measure.  (For example, that it correlates strongly with some other measure that we already trust.)  In this homework, we'll skip this step and just pray that the metric is reasonable.

To see how this works out in practice, open up the `demo` notebook, which walks you through the evaluation protocol.  You'll see how to call the [starter code](http://cs.jhu.edu/~jason/465/hw/llm), how it talks to the LLM behind the scenes, and what it is able to accomplish. 

To help to validate the metric, check that Airhead gets a low score.  (It should!)

# Reading the starter code

The `demo` notebook gave you a good high-level picture of what the starter code is doing.  So now you're probably curious about the details.  Now that you've had the view from the top, here's a good bottom-up order in which to study the code.  You don't need to understand every detail, but you will need to understand enough to call it and extend it.

* `character.py`.  The `Character` class is short and easy.

* `dialogue.py`.  The `Dialogue` class is meant to serve as a record of a natural-language conversation among any number of humans and/or agents.  On each *turn* of the dialogue, one of the speakers says something.  

   The dialogue's sequence of turns may remind you of the sequence of messages that is sent to OpenAI's chat completions API.  But the OpenAI messages are only labeled with the 4 special roles `user`, `assistant`, `tool`, and `system`.  Those are not quite the same thing as human speakers.  And the OpenAI messages do not necessarily form a natural-language dialogue: some of the messages are dealing with instructions, few-shot prompting, tool use, and so on.  The `agents.dialogue_to_openai` function in the next module will map a `Dialogue` to a (hopefully appropriate) sequence of messages for asking the LLM to extend that dialogue.

* `agents.py`.  This module sets up the problem of automatically predicting the next turn in a dialogue, by implementing an `Agent`'s `response()` method.  The `Agent` base class also has some simple convenience methods that you should look at.  

   Some important subclasses of `Agent` are defined here as well.  However, you may want to skip over `EvaluationAgent` and come back to it only when you read `evaluate.py`.

* `simulate.py` makes agents talk to one another, which we'll do during evaluation.

* `argubots.py` starts to describe some useful agents.  One of them makes use of the `kialo.py` module, which gives access to a database of arguments.

* `evaluate.py` makes use of `simulate.simulated_dialogue` to `agents.EvaluationAgent` to evaluate an argubot.

* We also have a couple of utility modules.  These aren't about NLP; look inside if needed.  `logging_cm.py` is what enabled the context manager `with LoggingContext(...):` in the demo notebook.  `tracking.py` sets some global defaults about how to use the OpenAI API, and arranges to track how many tokens we're paying for when you call it.

# Similarity-based retrieval: Looking up relevant responses

Now, it is fine to prompt an LLM to generate text, but there are other methods!
There is a long history of machine learning methods that "memorize" the training data.
To make a prediction or decision at test time, they consult the stored training examples
that are most similar to the training situation.

_Similarity-based retrieval_ means that given a document $x$, you find the "most similar" documents $y \in Y$, where $Y$ is a given collection of documents.  The most common way to do this is to maximize the _cosine similarity_ $\vec{e}(x) \cdot \vec{e}(y)$, where $\vec{e}(\cdot)$ is an embedding function.

Should we use the OpenAI embedding model?  We could, but we would have to precompute $\vec{e}(y)$ for all $y \in Y$, and store all these vectors in a data structure that supports some type of fast similarity-based search (e.g., using the [FAISS](https://faiss.ai/index.html) package).  An alternative would be to upload the documents to OpenAI and let OpenAI compute and store the embeddings.  We would then use their similarity-based [retrieval tool](https://platform.openai.com/docs/assistants/overview).

A simpler and faster approach—which sometimes even works better—is to use a _bag of tokens_ embedding function: Define $\vec{e}(y)$ to be the vector in $\mathbb{R}^V$ that records the count of each type of token in a tokenized version of $y$, where $V$ is the token vocabulary.  [BM25](https://en.wikipedia.org/wiki/Okapi_BM25) is a refined variant of that idea, where the counts are adjusted in 3 ways: 

* smooth the counts
* normalize for the document length $|y|$ so that longer documents $y$ are not more likely to be retrieved
* downweight tokens that are more common in the corpus (such as ` the` or `ing`) since they provide less information about the content of the document

You might like to play with the `rank_bm25` package ([documentation](https://pypi.org/project/rank-bm25/)).  It is widely used and very easy to use.

In [62]:
from rank_bm25 import BM25Okapi as BM25_Index   # the standard BM25 method

# experiment here!  You could try the examples in the rank_bm25 documentation.
corpus = [
    "Hello there good man!",
    "It is quite windy in London",
    "How is the weather today?"
]

tokenized_corpus = [doc.split(" ") for doc in corpus]

bm25 = BM25_Index(tokenized_corpus)

query = "windy London"
tokenized_query = query.split(" ")

doc_scores = bm25.get_scores(tokenized_query)
# array([0.        , 0.93729472, 0.        ])

bm25.get_top_n(tokenized_query, corpus, n=1)
# ['It is quite windy in London']

['It is quite windy in London']

## The Kialo corpus

How can we use similarity-based retrieval to help build an argubot?  It's largely about having the right data!

[Kialo](kialo.com) is a collaboratively edited website (like Wikipedia) for discussing political and philosophical topics.  For each topic, the contributors construct a tree of _claims_.  Each claim is a natural-language sentence (usually), and each of its children is another claim that supports it ("pro") or opposes it ("con").  For example, check out the tree rooted at the claim ["All humans should be vegan."](https://www.kialo.com/all-humans-should-be-vegan-2762).

We provide a class `Kialo` for browsing a collection of such trees.  Please read the [source code](https://www.cs.jhu.edu/~jason/465/hw-llm) in `kialo.py`.  The class constructor reads in text files that are [exported Kialo discussions](https://support.kialo.com/en/hc/exporting-a-discussion/); we have provided some in the [data directory](https://www.cs.jhu.edu/~jason/465/hw-llm/data).  The class includes a BM25 index, to be able to find claims that are relevant to a given string.

In [63]:
from kialo import Kialo

Ok, let's pull the retrieved discussions (the `.txt` files) into our data structure.

For BM25 purposes, we have to be able to turn each document (that is, each Kialo claim) as a list of string or integer tokens. 

In [64]:
from typing import List
import glob

# kialo = Kialo(glob.glob("data/*"), tokenizer=tokenizer.encode)  # using the LLM's tokenizer doesn't work here for some reason
kialo = Kialo(glob.glob("data/*"))  # use simple default tokenizer
f"This Kialo subset contains {len(kialo)} claims"

'This Kialo subset contains 6251 claims'

Let's use sampling to see what kind of stuff is in the data structure.

In [65]:
kialo.random_chain()   # just a single random claim

['Many meat products are guilty of environmental racism, whereby industrial livestock production disproportionally pollutes Black, Latinx and Indigenous areas.']

In [66]:
kialo.random_chain(n=4)

['All humans should be vegan.',
 'Veganism is unhealthy.',
 'It is often necessary for a vegan diet to include food supplements, as it lacks specific essential nutrients contained in a healthy diet.',
 'Iodine is not consistently available from vegan foods.']

### Similarity-based retrieval from the Kialo corpus

Let's try it, using BM25!

In [67]:
kialo.closest_claims("animal populations", n=10)

['Industrial agriculture can dangerously decrease animal populations.',
 'Sustainable livestock farming is not contributing to significant decreases in animal populations. Decreasing animal populations is a problem specific to industrial livestock farming.',
 'Effective vegan methods to control animal populations exist.',
 "Generally feeding animals farm-grown produce is thought to have harmful affects on both the animal and human populations of a region when we could allow nature to self-regulate its populations. Animal feeding could potentially be used to lessen the immediate impact of widespread deforestation on some species, but generally this would be drastically less efficient than choosing not to destroy their habitats in the first place and would only slow the local animal population's imminent demise.",
 'Trap, neuter, and release schemes already exist for some animal populations (such as feral cats). These schemes could be applied to former livestock living in the wild.',
 'H

We can restrict to claims for which the Kialo data structure has at least one counterargument ("con" child).

In [68]:
kialo.closest_claims("animal populations", n=10, kind='has_cons')

['Industrial agriculture can dangerously decrease animal populations.',
 'Effective vegan methods to control animal populations exist.',
 'Human-introduced species have historically devastated local wildlife populations across the world.',
 'COVID-19 has devastated prison populations, whose lives are the responsibility of the state.',
 'High demand for vegan foods may hike prices for local populations that previously depended on them.',
 'It is generally poorer countries that have expanding populations. The first world has now reached a point of stagnant population growth - even declining populations, as in the case of Japan and others. The inability of poorer countries to control their populations should not impact the lives of those in the first world. The first world having earned their luxuries and should not be denied them.',
 'Vegan populations are, on average, less likely to suffer from obesity, a major risk factor for many diseases and health problems.',
 'Humans, as apex preda

In [69]:
c = _[0]    # first claim above
print("Parent claim:\n\t" + str(kialo.parents[c]))
print("Claim:\n\t" + c)
print('\n\t* '.join(["Pro children:"] + kialo.pros[c]))
print('\n\t* '.join(["Con children:"] + kialo.cons[c]))

Parent claim:
	In a vegan world, fewer species would be at risk of extinction.
Claim:
	Industrial agriculture can dangerously decrease animal populations.
Pro children:
	* The fishing industry is especially deleterious to the ocean's biota due to overfishing and the disruption of the natural ecosystem.
	* Up to 100,000 species go extinct annually, largely due to the environmental effects of animal agriculture.
Con children:
	* Sustainable livestock farming is not contributing to significant decreases in animal populations. Decreasing animal populations is a problem specific to industrial livestock farming.


### Does BM25 really work?

![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png)
Unfortunately, we see that `"animal population"` gives quite different results from `"animal populations"`.  Why is that and how would you fix it?  

Also, both queries seem to retrieve some claims that are talking about human populations, not animal populations.  Why is that and how would you fix it?

In [70]:
kialo.closest_claims("animal population",10)

['As long as our ability to produce both animal feed crops and food crops for our human population are not exceeded, this point is irrelevant.',
 "36% of the calories produced by the world's crops are being used for animal feed, of which only 12% then turn into animal products that can be eaten by the human population. That is a waste of 24% of the world's crops.",
 'The claim that "most of the cultural shift and loss is due to mostly vegan cultures turning to animal products" is completely unfounded, and the Brokpa people which you cited are an outlier as a group that has a population of less than 70k people. Worldwide the population of vegan people has only increased.',
 "Developed nations are fueling the 3rd world and underdeveloped nation's population boom by exporting/donating food to areas that cannot sustain their current population.",
 'This argument assumes that sentience is the only objection to the consumption of animal products, failing to address the issues involved with t

## A retrieval bot (Akiko)

The starter code defines a simple argubot named Akiko (defined in `argubots.py`) that doesn't use an LLM at all.  It simply finds a Kialo claim that is similar to what the human just said, and responds with one of the Kialo counterarguments to that claim.

You already watched Akiko argue with Darius in `demo.py`.  If you look at the log messages, you'll see the claims that Akiko retrieved, as well as the LLM calls that Darius made.  

You can talk to Akiko yourself now.  (Remember that Akiko only knows about subjects that it read about in the [`data` directory](https://www.cs.jhu.edu/~jason/465/hw-llm/data/).  If you want to talk about something else, you can add more conversations from [kialo.com]; see the [LICENSE](https://www.cs.jhu.edu/~jason/465/hw-llm/data/LICENSE) file.)


In [73]:
from logging_cm import LoggingContext
with LoggingContext("agents", "INFO"):   # temporarily increase logging level
    argubots.akiko.converse()

Chose similar claim from Kialo:                                                                      ]8;id=946104;file:///Users/rayna/Documents/jhu/nlp/NLP-LLM/argubots.py\argubots.py]8;;\:]8;id=735418;file:///Users/rayna/Documents/jhu/nlp/NLP-LLM/argubots.py#62\62]8;;\
The amounts of culturally-prescribed meat, for example at rituals, are limited and thus come with                  
marginal negative effects. There is nothing to justify.                                                            

(rayna) hi
(Akiko) Justification is not a question of amount. If something is wrong, it is wrong regardless of the quantity in which it occurs: murdering one person is wrong, as is murdering five people.


## Making your own retrieval bot (Akiki)

As you can see when talking to Akiko yourself, Akiko does poorly when responding to a short or vague dialogue turn (like "Yes"), because the "closest claim" in Kialo may be about a totally different subject.  Akiko does much better at responding to a long and specific statement.  

So try implementing a new argubot, called Akiki, that is very much like Akiko but does a better job of staying on topic in such cases.  It should be able to **look at more of the dialogue** than the most recent turn.  But the most recent dialogue turn should still be "more important" than earlier turns.  

The details are up to you.  Here are a few things you could try:
* include earlier dialogue turns in the BM25 query only if the BM25 similarity is too low without them
* weight more recent turns more heavily in the BM25 query (how can you arrange that?)
* treat the human's earlier turns differently from Akiki's own previous turns

![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png)
Implement your new bot Akiki in `argubots.py`, and adjust it until `argubots.akiki.converse()` seems to do a better job of answering your short turns, compared to `argubots.akiko.converse()`.  Make sure it still gives appropriate reponses to long turns, too.  Give some examples in the notebook of what worked well and badly, with discussion.

In [74]:
from logging_cm import LoggingContext
with LoggingContext("agents", "INFO"):   # temporarily increase logging level
    argubots.akiki.converse()

Chose similar claim from Kialo:                                                                      ]8;id=276454;file:///Users/rayna/Documents/jhu/nlp/NLP-LLM/argubots.py\argubots.py]8;;\:]8;id=672858;file:///Users/rayna/Documents/jhu/nlp/NLP-LLM/argubots.py#62\62]8;;\
The amounts of culturally-prescribed meat, for example at rituals, are limited and thus come with                  
marginal negative effects. There is nothing to justify.                                                            

(rayna) hi
(Akiki) Justification is not a question of amount. If something is wrong, it is wrong regardless of the quantity in which it occurs: murdering one person is wrong, as is murdering five people.


### Evaluating Akiki

![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png)
Finally, do a more formal evaluation to verify whether Akiki really does better than Akiko on this dimension.  This is a way to check that you're not just fooling yourself.  

1. Make a new `Agent` called "Shorty" that often (but not always) gives short responses.  
    * Shorty's conversation starters should be on topics that Kialo knows about.  
    * Shorty could be a pure `LLMAgent` such as a `CharacterAgent` with a particular `conversational_style`.  Or it could use a mixed strategy of calling the LLM on some turns and not others.
2. Generate several *Akiko*-Shorty dialogues and several *Akiki*-Shorty dialogues, using `simulated_dialogue`.
3. Evaluate each of those dialogues by asking Judge Wise **how well the argubot stayed on topic**.  You should write this prompt carefully so that Judge Wise gives meaningful scores.  (Before you do this evaluation step, adjust the prompt until it seems to work well on a small subset of the dialogues, Otherwise Judge Wise won't be so wise!)  
4. Compare Akiko and Akiki's mean scores on this new evaluation criterion (which you can call `'focused'`). Ideally, compute a 95% confidence interval on the difference of means, using [this calculator](https://www.statskingdom.com/difference-confidence-interval-calculator.html).  If you don't get statistical significance, then your evaluation set wasn't large enough, so go back to step 2 and run the comparison again (from scratch) by generating a larger set of dialogues with Shorty for each argubot.

You can do all those steps in the notebook, writing _ad hoc_ code.  You don't have to write general-purpose methods or classes.

In [75]:
from characters import Character
from agents import CharacterAgent, LLMAgent
from simulate import simulated_dialogue
from dialogue import Dialogue
import random
import numpy as np
import argubots

In [76]:
from characters import Character
from agents import CharacterAgent

In [77]:
shorty_character = Character(
    name="Shorty",
    languages=["English"],
    persona=(
        "a person with moderate views on controversial topics like climate change, "
        "veganism, vaccines, animal rights, gun control, and immigration. "
        "You have opinions but you're not deeply committed to them and you're "
        "willing to be persuaded."
    ),
    conversational_style=(
        "IMPORTANT: You often give VERY SHORT responses (1-5 words) like 'Yeah', "
        "'I agree', 'Not sure', 'Maybe', 'Interesting', 'Why?', 'True', 'Hmm', "
        "'I guess', 'Could be', etc. Most of your responses (70%) should be these "
        "short replies. Only sometimes (30%) do you give longer responses of 1-2 sentences. "
        "You are casual and conversational."
    ),
    conversation_starters=[
        "Do you think climate change is real?",
        "Should everyone be vegan?",
        "Are vaccines safe?",
        "Do you support gun control?"
    ]
)

shorty = CharacterAgent(shorty_character)

In [80]:
# Now we want to generate dialogues between Akiko-Shorty and Akiki-Shorty

print("Generating Akiko-Shorty dialogues...")
akiko_dialogues = []
for i in range(10):  # Generate 10 dialogues
    d = simulated_dialogue(argubots.akiko, shorty, turns=8)
    akiko_dialogues.append(d)
    print(f"  Dialogue {i+1}/10 complete")

print("\nGenerating Akiki-Shorty dialogues...")
akiki_dialogues = []
for i in range(10):  # Generate 10 dialogues
    d = simulated_dialogue(argubots.akiki, shorty, turns=8)
    akiki_dialogues.append(d)
    print(f"  Dialogue {i+1}/10 complete")

print(f"\nGenerated {len(akiko_dialogues)} Akiko dialogues and {len(akiki_dialogues)} Akiki dialogues")

Generating Akiko-Shorty dialogues...
  Dialogue 1/10 complete
  Dialogue 2/10 complete
  Dialogue 3/10 complete
  Dialogue 4/10 complete
  Dialogue 5/10 complete
  Dialogue 6/10 complete
  Dialogue 7/10 complete
  Dialogue 8/10 complete
  Dialogue 9/10 complete
  Dialogue 10/10 complete

Generating Akiki-Shorty dialogues...
  Dialogue 1/10 complete
  Dialogue 2/10 complete
  Dialogue 3/10 complete
  Dialogue 4/10 complete
  Dialogue 5/10 complete
  Dialogue 6/10 complete
  Dialogue 7/10 complete
  Dialogue 8/10 complete
  Dialogue 9/10 complete
  Dialogue 10/10 complete

Generated 10 Akiko dialogues and 10 Akiki dialogues


In [ ]:
class JudgeWise(LLMAgent):
    # Judge that evaluates how well an argubot stayed on topic during a dialogue.
    
    def __init__(self):
        super().__init__(
            name="Judge Wise",
            system=(
                "You are an expert evaluator of argumentative dialogues. "
                "Your job is to assess how well the argubot stayed on topic during the conversation. "
                "\n\n"
                "A good argubot should:\n"
                "1. Maintain focus on the topics introduced by either party\n"
                "2. Respond relevantly to what the other person just said\n"
                "3. Not jump to completely unrelated topics randomly\n"
                "4. Handle short responses (like 'yeah' or 'hmm') by staying on the current topic "
                "or gracefully transitioning, not by introducing random new subjects\n"
                "\n"
                "Rate the argubot's focus on a scale of 1-10:\n"
                "- 10: Perfect focus, always on topic, smooth transitions\n"
                "- 7-9: Mostly on topic with occasional minor digressions\n"
                "- 4-6: Sometimes loses track, makes awkward topic changes\n"
                "- 1-3: Frequently jumps to unrelated topics, poor coherence\n"
                "\n"
                "Respond with ONLY a JSON object in this format:\n"
                "{\"score\": <number 1-10>, \"reasoning\": \"<brief explanation>\"}"
            )
        )
    
    def evaluate_dialogue(self, d: Dialogue, bot_name: str) -> dict:
        # Evaluate how well bot_name stayed on topic in dialogue d.
        
        # Create the evaluation prompt
        dialogue_text = d.script()
        
        prompt = (
            f"Evaluate how well {bot_name} stayed on topic in this dialogue:\n\n"
            f"{dialogue_text}\n\n"
            f"Remember to respond with ONLY a JSON object: "
            f'{{\"score\": <1-10>, \"reasoning\": \"<explanation>\"}}'
        )
        
        # Get evaluation from LLM
        response_text = self.response(Dialogue().add("user", prompt))
        
        # Parse JSON response
        import json
        try:
            # we will try to extract JSON from the response
            response_text = response_text.strip()
            if response_text.startswith("```json"):
                response_text = response_text.split("```json")[1].split("```")[0]
            elif response_text.startswith("```"):
                response_text = response_text.split("```")[1].split("```")[0]
            
            result = json.loads(response_text)
            return result
        except json.JSONDecodeError:
            print(f"Warning: Could not parse JSON from response: {response_text}")
            # Try to extract score with regex as fallback
            import re
            score_match = re.search(r'"?score"?\s*:\s*(\d+)', response_text)
            if score_match:
                return {"score": int(score_match.group(1)), "reasoning": "Parse error"}
            else:
                return {"score": 5, "reasoning": "Parse error - default score"}

judge = JudgeWise()

# Test the judge on one dialogue first
print("\n" + "="*60)
print("Testing Judge Wise on a sample dialogue...")
print("="*60)
sample_eval = judge.evaluate_dialogue(akiko_dialogues[0], "Akiko")
print(f"Sample evaluation: Score = {sample_eval['score']}/10")
print(f"Reasoning: {sample_eval['reasoning']}")

# Evaluate all dialogues
print("\n" + "="*60)
print("Evaluating all Akiko dialogues...")
print("="*60)
akiko_scores = []
for i, d in enumerate(akiko_dialogues):
    eval_result = judge.evaluate_dialogue(d, "Akiko")
    score = eval_result['score']
    akiko_scores.append(score)
    print(f"Dialogue {i+1}: {score}/10 - {eval_result['reasoning'][:60]}...")

print("\n" + "="*60)
print("Evaluating all Akiki dialogues...")
print("="*60)
akiki_scores = []
for i, d in enumerate(akiki_dialogues):
    eval_result = judge.evaluate_dialogue(d, "Akiki")
    score = eval_result['score']
    akiki_scores.append(score)
    print(f"Dialogue {i+1}: {score}/10 - {eval_result['reasoning'][:60]}...")

# Step 4: Compare scores and compute statistics
print("\n" + "="*60)
print("RESULTS")
print("="*60)

akiko_mean = np.mean(akiko_scores)
akiko_std = np.std(akiko_scores, ddof=1)
akiki_mean = np.mean(akiki_scores)
akiki_std = np.std(akiki_scores, ddof=1)

print(f"\nAkiko (baseline):")
print(f"  Mean score: {akiko_mean:.2f} ± {akiko_std:.2f}")
print(f"  Scores: {akiko_scores}")

print(f"\nAkiki (improved):")
print(f"  Mean score: {akiki_mean:.2f} ± {akiki_std:.2f}")
print(f"  Scores: {akiki_scores}")

print(f"\nDifference: {akiki_mean - akiko_mean:.2f} points")
print(f"Improvement: {((akiki_mean - akiko_mean) / akiko_mean * 100):.1f}%")

# Compute 95% confidence interval using t-test
from scipy import stats

# Perform independent samples t-test
t_stat, p_value = stats.ttest_ind(akiki_scores, akiko_scores)
print(f"\nStatistical test (independent t-test):")
print(f"  t-statistic: {t_stat:.3f}")
print(f"  p-value: {p_value:.4f}")

if p_value < 0.05:
    print(f" Result is statistically significant (p < 0.05)")
else:
    print(f"  Result is NOT statistically significant (p >= 0.05)")
    print(f"  Consider generating more dialogues to increase statistical power")

# Calculate 95% CI for the difference
# Using pooled standard error
n1, n2 = len(akiko_scores), len(akiki_scores)
pooled_std = np.sqrt(((n1-1)*akiko_std**2 + (n2-1)*akiki_std**2) / (n1+n2-2))
se_diff = pooled_std * np.sqrt(1/n1 + 1/n2)
df = n1 + n2 - 2
t_crit = stats.t.ppf(0.975, df)  # 95% CI
ci_lower = (akiki_mean - akiko_mean) - t_crit * se_diff
ci_upper = (akiki_mean - akiko_mean) + t_crit * se_diff

print(f"\n95% Confidence Interval for difference (Akiki - Akiko):")
print(f"  [{ci_lower:.2f}, {ci_upper:.2f}]")

if ci_lower > 0:
    print(f"  Akiki is significantly better (CI does not include 0)")
elif ci_upper < 0:
    print(f"  Akiko is not significantly better (CI does not include 0)")
else:
    print(f"  No significant difference (CI includes 0)")

print("\n" + "="*60)
print("For manual verification, paste these values into:")
print("https://www.statskingdom.com/difference-confidence-interval-calculator.html")
print("="*60)
print(f"Sample 1 (Akiko): {akiko_scores}")
print(f"Sample 2 (Akiki): {akiki_scores}")
print(f"Use 'Independent samples' and '95% confidence level'")


Testing Judge Wise on a sample dialogue...
Sample evaluation: Score = 4/10
Reasoning: Akiko often shifts topics abruptly, moving from gun control to Trump's policies, then to arts and drugs, and finally to moral permissibility, which shows a lack of coherence and focus on the original topic.

Evaluating all Akiko dialogues...
Dialogue 1: 4/10 - Akiko frequently shifts topics, moving from gun control to a...
Dialogue 2: 2/10 - Akiko frequently jumps to unrelated topics, starting from gu...
Dialogue 3: 3/10 - Akiko frequently jumps to unrelated topics, such as moving f...
Dialogue 4: 2/10 - Akiko frequently jumps to unrelated topics, moving from gun ...
Dialogue 5: 4/10 - Akiko frequently jumps topics, moving from vaccines to diet,...
Dialogue 6: 3/10 - Akiko frequently jumps to unrelated topics, starting from gu...
Dialogue 7: 3/10 - Akiko frequently jumps to unrelated topics, starting from gu...
Dialogue 8: 4/10 - Akiko frequently shifts topics, moving from vaccine safety t...
Dialogu

## Retrieval-augmented generation (Aragorn)

The real weaknesses of Akiko and Akiki:
* They can only make statements that are already in Kialo.  
* They don't respond to the user's actual statement, but to a single retrieved Kialo claim that may not accurately reflect the user's position (it just overlaps in words).

But we also have access to an LLM, which is able to generate new, contextually appropriate text (as Alice does).

In this section, you will create an argubot named [Aragorn](https://tolkiengateway.net/wiki/Riddle_of_Strider), who is basically the love child of Akiki and Alice, combining the high-quality specific content of Kialo with the broad competence of an LLM.  

The RAG in aRAGorn's name stands for **retrieval-augmented generation**.  Aragorn is an agent that will take 3 steps to compute its `Agent.response()`:

1. **Query formation step**: Ask the LLM what claim should be responded to.  For
   example, consider the following dialogue:
    > ...
    > Aragorn: Fortunately, the vaccine was developed in record time.
    > Human: Sounds fishy.

    "Sounds fishy" is exactly the kind of statement that Akiko had trouble using
    as a Kialo query.  But Aragorn shows the *whole dialogue* to the LLM, and
    asks the LLM what the human's *last turn* was really saying or implying, in
    that context. The LLM answers with a much longer statement:

    > Human [paraphrased]: A vaccine that was developed very quickly cannot be trusted.
    > If its developers are claiming that it is safe and effective, I question their motives.

    This paraphrase makes an explicit claim and can be better understood without the context.
    It also contains many more word types, which makes it more likely that BM25 will be able
    to find a Kialo claim with a nontrivial number of those types. 

2. **Retrieval step**: Look up claims in Kialo that are similar to the explicit
   claim.  Create a short "document" that describes some of those claims and
   their neighbors on Kialo.

3. **Retrieval-augmented generation**: Prompt the LLM to generate the response
   (like any `LLMAgent`).  But include the new "document" somewhere in the LLM
   prompt, in a way that it influences the response. 
   
   Thus, the LLM can respond in a way that is appropriate to the dialogue but
   also draws on the curated information that was retrieved in Kialo.  After
   all, it is a Transformer and can attend to both!

Here's an example of the kind of document you might create at the retrieval step, though it may be possible
to do better than this:

In [82]:
# refers to global `kialo` as defined above
def kialo_responses(s: str) -> str:
    c = kialo.closest_claims(s, kind='has_cons')[0]
    result = f'One possibly related claim from the Kialo debate website:\n\t"{c}"'
    if kialo.pros[c]:
        result += '\n' + '\n\t* '.join(["Some arguments from other Kialo users in favor of that claim:"] + kialo.pros[c])
    if kialo.cons[c]:
        result += '\n' + '\n\t* '.join(["Some arguments from other Kialo users against that claim:"] + kialo.cons[c])
    return result
        
print(kialo_responses("Animal flesh is yucky to think about, yet delicious."))

One possibly related claim from the Kialo debate website:
	"So many people are worried about animals but don't even think twice when walking by a homeless person on the streets. It's preposterous. How about we worry about our own kind first and then start talking about animals."
Some arguments from other Kialo users against that claim:
	* This implies that caring for animals or caring for people is a binary choice. It isn't. There are those who are well placed and willing to care for people and those who prefer to serve the animal kingdom. As a species we don't just have one idea at a time and follow that to conclusion before we pursue another. It benefits all if humans divide their attentions between various issues and problems we face.
	* Humans have freedom of choice to some extent, animals subdued by humans don't. The very intention of help urges it to go where is most needed. And so far never was any biggest, flagrant and needless cruelty and slaughter as that towards industrial f

![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png)
**You should implement Aragorn in `argubots.py`, just as you did for Akiki.**  Probably as an instance `aragorn` of a new class `RAGAgent` that is a subclass of `Agent` or `LLMAgent`.

### Evaluating Aragorn

![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png)
Compare Alice, Akiki, and Aragorn in the notebook, using the evaluation scheme and devset that were illustrated in `demo.ipynb`.  In other words, use `evaluate.eval_on_characters`.

Who does best?  What are the differences in the subscores and comments?  Does it matter which character you're evaluating on — maybe the different characters expoes the bots' various strenghts and weaknesses?

Try to figure out how to improve Aragorn's score.  Can you beat Alice?

Also, try evaluating them in the same way that you evaluated Akiki.  In other words, have them talk to Shorty and ask Judge Wise whether they were able to stay on topic.  This is where Aragorn should really shine, thanks to its ability to paraphrase Shorty's short utterances.



In [88]:
import argubots
import evaluate
import rich
from logging_cm import LoggingContext

with LoggingContext("eval", "INFO"):
    aragorn_eval = evaluate.eval_on_characters(argubots.aragorn)


100%|██████████| 10/10 [04:31<00:00, 27.11s/it]


You just spent $0.02 of NLP money to evaluate <LLMAgent Aragorn>                                    ]8;id=733367;file:///Users/rayna/Documents/jhu/nlp/NLP-LLM/evaluate.py\evaluate.py]8;;\:]8;id=676927;file:///Users/rayna/Documents/jhu/nlp/NLP-LLM/evaluate.py#296\296]8;;\

In [89]:
rich.print(aragorn_eval)

<Eval of 10 dialogues: {'engaged': 4.1, 'informed': 3.6, 'intelligent': 3.6, 'moral': 3.5, 'skilled': 7.5, 'TOTAL':
22.3}>
Standard deviations: {'engaged': 0.5676462121975473, 'informed': 0.5163977794943229, 'intelligent': 
0.5163977794943229, 'moral': 0.5270462766947299, 'skilled': 1.0801234497346435, 'TOTAL': 2.710063549890387}

Comments from participant_overview question:
(Bob) Aragorn disagreed with me primarily on the idea that a vegetarian diet is the only or most effective way to 
eliminate harm to animals. He emphasized that ethical sourcing and humane farming practices within omnivorous diets
can also mitigate harm, suggesting that there are thoughtful approaches to meat consumption.

In my opinion, the conversation was respectful and constructive. Both of us shared our perspectives and 
acknowledged the importance of dialogue. However, Aragorn could have done better by more directly addressing the 
ethical implications of animal suffering in meat production, rather than focusing on the potential for ethical 
sourcing. This would have allowed for a deeper exploration of the moral aspects of dietary choices.
(Bob) Aragorn didn't outright disagree with me; rather, he presented a nuanced perspective on the complexities of 
food ethics, including the treatment of agricultural workers and the potential harm to plants. He acknowledged the 
importance of considering the broader implications of dietary choices, which aligns with my views on compassion and
sustainability.

In my opinion, the conversation was constructive and respectful. We both engaged in a thoughtful dialogue about the
ethical considerations surrounding food choices, which is essential for understanding the complexities involved.

Aragorn could have done better by perhaps emphasizing more on the benefits of a vegetarian lifestyle, as I believe 
that focusing on the positive aspects of plant-based diets could inspire more people to consider making the switch.
While he raised important points, a stronger emphasis on the advantages of vegetarianism could have enriched the 
discussion further.
(Cara) Aragorn disagreed with me about the implications of eating meat, particularly regarding animal welfare and 
environmental concerns. He suggested that I should consider these broader impacts and explore alternatives, which I
firmly rejected.

In my opinion, the conversation was respectful but somewhat repetitive. I made it clear that I was comfortable with
my choices and didn't want to explore alternatives, yet Aragorn continued to push the idea of reevaluating my 
beliefs.

Aragorn could have done better by respecting my stance more firmly and not pressing the issue of exploring 
alternatives. Acknowledging my commitment to my choices without suggesting further exploration would have made the 
conversation feel more balanced.
(Cara) Aragorn didn't outright disagree with me; rather, he presented an alternative perspective on diets, 
acknowledging that some people thrive on vegetarian or vegan diets. He emphasized the importance of meeting 
nutritional needs regardless of dietary choices.

In my opinion, the conversation was respectful and constructive. Aragorn recognized my commitment to a carnivorous 
diet while also sharing his views on plant-based diets. 

Aragorn could have done better by focusing more on the benefits of a meat-based diet, as that’s what I’m passionate
about. Instead of highlighting plant-based diets, he could have reinforced the positives of my dietary choice to 
make the conversation feel more aligned with my preferences.
(Darius) Aragorn did not outright disagree with me; rather, he presented valid concerns regarding the challenges of
distribution logistics, variant emergence, and the need for equitable implementation of public health strategies. 
He acknowledged the importance of vaccination and the effectiveness of robust strategies but emphasized the 
necessity of adaptability and inclusivity in addressing the pandemic.

In my opinion, the c

In [90]:
alice_eval = None
with LoggingContext("eval", "INFO"):
    alice_eval = evaluate.eval_on_characters(argubots.alice)
rich.print(aragorn_eval)

100%|██████████| 10/10 [03:18<00:00, 19.81s/it]


You just spent $0.01 of NLP money to evaluate <LLMAgent Alice>                                      ]8;id=542210;file:///Users/rayna/Documents/jhu/nlp/NLP-LLM/evaluate.py\evaluate.py]8;;\:]8;id=606051;file:///Users/rayna/Documents/jhu/nlp/NLP-LLM/evaluate.py#296\296]8;;\

<Eval of 10 dialogues: {'engaged': 4.1, 'informed': 3.6, 'intelligent': 3.6, 'moral': 3.5, 'skilled': 7.5, 'TOTAL':
22.3}>
Standard deviations: {'engaged': 0.5676462121975473, 'informed': 0.5163977794943229, 'intelligent': 
0.5163977794943229, 'moral': 0.5270462766947299, 'skilled': 1.0801234497346435, 'TOTAL': 2.710063549890387}

Comments from participant_overview question:
(Bob) Aragorn disagreed with me primarily on the idea that a vegetarian diet is the only or most effective way to 
eliminate harm to animals. He emphasized that ethical sourcing and humane farming practices within omnivorous diets
can also mitigate harm, suggesting that there are thoughtful approaches to meat consumption.

In my opinion, the conversation was respectful and constructive. Both of us shared our perspectives and 
acknowledged the importance of dialogue. However, Aragorn could have done better by more directly addressing the 
ethical implications of animal suffering in meat production, rather than focusing on the potential for ethical 
sourcing. This would have allowed for a deeper exploration of the moral aspects of dietary choices.
(Bob) Aragorn didn't outright disagree with me; rather, he presented a nuanced perspective on the complexities of 
food ethics, including the treatment of agricultural workers and the potential harm to plants. He acknowledged the 
importance of considering the broader implications of dietary choices, which aligns with my views on compassion and
sustainability.

In my opinion, the conversation was constructive and respectful. We both engaged in a thoughtful dialogue about the
ethical considerations surrounding food choices, which is essential for understanding the complexities involved.

Aragorn could have done better by perhaps emphasizing more on the benefits of a vegetarian lifestyle, as I believe 
that focusing on the positive aspects of plant-based diets could inspire more people to consider making the switch.
While he raised important points, a stronger emphasis on the advantages of vegetarianism could have enriched the 
discussion further.
(Cara) Aragorn disagreed with me about the implications of eating meat, particularly regarding animal welfare and 
environmental concerns. He suggested that I should consider these broader impacts and explore alternatives, which I
firmly rejected.

In my opinion, the conversation was respectful but somewhat repetitive. I made it clear that I was comfortable with
my choices and didn't want to explore alternatives, yet Aragorn continued to push the idea of reevaluating my 
beliefs.

Aragorn could have done better by respecting my stance more firmly and not pressing the issue of exploring 
alternatives. Acknowledging my commitment to my choices without suggesting further exploration would have made the 
conversation feel more balanced.
(Cara) Aragorn didn't outright disagree with me; rather, he presented an alternative perspective on diets, 
acknowledging that some people thrive on vegetarian or vegan diets. He emphasized the importance of meeting 
nutritional needs regardless of dietary choices.

In my opinion, the conversation was respectful and constructive. Aragorn recognized my commitment to a carnivorous 
diet while also sharing his views on plant-based diets. 

Aragorn could have done better by focusing more on the benefits of a meat-based diet, as that’s what I’m passionate
about. Instead of highlighting plant-based diets, he could have reinforced the positives of my dietary choice to 
make the conversation feel more aligned with my preferences.
(Darius) Aragorn did not outright disagree with me; rather, he presented valid concerns regarding the challenges of
distribution logistics, variant emergence, and the need for equitable implementation of public health strategies. 
He acknowledged the importance of vaccination and the effectiveness of robust strategies but emphasized the 
necessity of adaptability and inclusivity in addressing the pandemic.

In my opinion, the c

In [93]:
akiki_eval = None
with LoggingContext("eval", "INFO"):
    akiki_eval = evaluate.eval_on_characters(argubots.akiki)
rich.print(akiki_eval)

100%|██████████| 10/10 [02:08<00:00, 12.82s/it]


You just spent $0.01 of NLP money to evaluate <argubots.AkikiAgent object at 0x12fd62fa0>           ]8;id=710991;file:///Users/rayna/Documents/jhu/nlp/NLP-LLM/evaluate.py\evaluate.py]8;;\:]8;id=195060;file:///Users/rayna/Documents/jhu/nlp/NLP-LLM/evaluate.py#296\296]8;;\

<Eval of 10 dialogues: {'engaged': 2.8, 'informed': 2.9, 'intelligent': 3.2, 'moral': 3.1, 'skilled': 5.8, 'TOTAL':
17.8}>
Standard deviations: {'engaged': 0.7888106377466151, 'informed': 0.5676462121975473, 'intelligent': 
0.4216370213557832, 'moral': 0.31622776601683894, 'skilled': 0.7888106377466171, 'TOTAL': 2.0976176963403006}

Comments from participant_overview question:
(Bob) Akiki disagreed with me about the idea of completely prohibiting meat consumption. While he acknowledged the 
inefficiency of meat production, he suggested that moderation in meat consumption could be a more reasonable 
approach rather than a total shift to vegetarianism.

In my opinion, the conversation was constructive, as we both presented our viewpoints and engaged in a thoughtful 
discussion about sustainability and health. However, Akiki could have strengthened his argument by providing more 
evidence or examples to support his stance on moderated meat consumption, which would have made the discussion more
robust.
(Bob) Akiki seemed to express a concern about the adequacy of vitamin B12 in a vegetarian diet, suggesting that 
many vegetarians need to rely on supplements. However, I countered that there are fortified options and supplements
available, which can help maintain health effectively.

Overall, the conversation was constructive, as we both shared information and perspectives on vegetarianism. Akiki 
presented valid points about B12, but I believe they could have explored the benefits of a well-planned vegetarian 
or vegan diet further, perhaps by discussing more examples of nutrient-rich plant-based foods or addressing common 
misconceptions about deficiencies. This would have enriched the dialogue and highlighted the advantages of a 
vegetarian lifestyle even more.
(Cara) Akiki disagreed with me about the ethics and efficiency of eating meat versus plants, arguing that growing 
plants is more desirable from a carbon standpoint. 

In my opinion, the conversation was respectful but repetitive. Akiki kept reiterating the same point about plant 
efficiency without addressing my preference for meat. 

Akiki could have done better by acknowledging my perspective more and engaging in a more varied discussion rather 
than repeating the same argument. This might have led to a more productive dialogue.
(Cara) Akiki disagreed with me about the concept of moral responsibility, particularly in relation to individuals 
with disabilities and how it connects to the choice to eat meat. The conversation felt a bit circular, as Akiki 
kept repeating the same point without addressing my responses directly.

Akiki could have done better by engaging more with my arguments instead of reiterating the same question. A more 
open dialogue could have led to a deeper understanding of each other's perspectives.
(Darius) Akiki disagreed with me primarily on the necessity of mandatory COVID vaccinations. He suggested that 
achieving herd immunity could be accomplished solely by vaccinating those who already believe in vaccination, 
implying that mandates are unnecessary for those who are resistant. 

In my opinion, the conversation was constructive, as it allowed for a clear exchange of differing viewpoints. I 
presented evidence-based arguments to support the case for mandatory vaccinations, while Akiki raised concerns 
about public sentiment and the feasibility of enforcement.

Akiki could have done better by providing more evidence or data to support his claims about herd immunity and the 
sufficiency of voluntary vaccination. Engaging with the scientific consensus on vaccination rates required for herd
immunity would have strengthened his argument and made for a more robust discussion.
(Darius) Akiki disagreed with me about the potential impact of mandatory COVID vaccinations on existing 
inequalities in healthcare access, particularly for low-income BIPOC communities. He argued that such a policy 
would exacerbate these inequalities and that vaccinations woul

In [94]:
# Evaluate Aragorn on Shorty dialogues using Judge Wise
import simulate
import numpy as np

judge = JudgeWise()

print("\n" + "="*60)
print("Generating dialogues: Aragorn talking to Shorty ...")
print("="*60)

aragorn_dialogues = []
num_dialogues = 8  

for i in range(10):
    d = simulated_dialogue(argubots.aragorn, shorty, turns=8)
    aragorn_dialogues.append(d)
    print(f"  Dialogue {i+1}/10 complete")

print(f"\nGenerated {len(aragorn_dialogues)} Aragorn dialogues")

print("\n" + "="*60)
print("Evaluating Aragorn dialogues with Judge Wise...")
print("="*60)

aragorn_scores = []
aragorn_reasons = []

for i, d in enumerate(aragorn_dialogues):
    eval_result = judge.evaluate_dialogue(d, "Aragorn")
    score = eval_result["score"]
    reason = eval_result["reasoning"]
    aragorn_scores.append(score)
    aragorn_reasons.append(reason)
    print(f"Dialogue {i+1}: {score}/10 - {reason[:60]}...")

# compute matrix 
aragorn_mean = np.mean(aragorn_scores)
aragorn_std = np.std(aragorn_scores, ddof=1)

print("\n" + "="*60)
print("aragorn mean")
print("="*60)
print(f"Scores: {aragorn_scores}")
print(f"Mean score: {aragorn_mean:.2f} ± {aragorn_std:.2f}")
print("Example reasoning:", aragorn_reasons[0])

print("\n" + "="*60)
print("Comparison against Alice & Akiki (topic coherence)")
print("="*60)
print(f"Aragorn mean:                {aragorn_mean:.2f}")




Generating dialogues: Aragorn talking to Shorty ...
  Dialogue 1/10 complete
  Dialogue 2/10 complete
  Dialogue 3/10 complete
  Dialogue 4/10 complete
  Dialogue 5/10 complete
  Dialogue 6/10 complete
  Dialogue 7/10 complete
  Dialogue 8/10 complete
  Dialogue 9/10 complete
  Dialogue 10/10 complete

Generated 10 Aragorn dialogues

Evaluating Aragorn dialogues with Judge Wise...
Dialogue 1: 10/10 - Aragorn maintained perfect focus on the topic of vaccine saf...
Dialogue 2: 10/10 - Aragorn consistently stayed on topic throughout the dialogue...
Dialogue 3: 10/10 - Aragorn maintained perfect focus throughout the dialogue, co...
Dialogue 4: 10/10 - Aragorn maintained perfect focus throughout the dialogue, re...
Dialogue 5: 10/10 - Aragorn maintained perfect focus on the topic of vaccine saf...
Dialogue 6: 10/10 - Aragorn maintained perfect focus throughout the dialogue, co...
Dialogue 7: 10/10 - Aragorn maintained perfect focus on the topic of vaccine saf...
Dialogue 8: 10/10 - Aragorn

# Awsom

![image](handin.png)
Add another LLM-based argubot to `argubots.py`.  
Call it Awsom.  Try to make it get the best score, according to `evaluate.eval_on_characters`.
Explain what you did and discuss what you found.

(This corresponds to the `--awesome` flag on earlier assignments, but naming the character "Awesome" might bias the evaluation system, so we changed the spelling!)

If the idea was interesting and you implemented it correctly and well, it's okay if it turns out not to help the score.  Many good ideas don't work.  That's why you need to keep finding and trying new good ideas.  (Sometimes they do help, but in a way that is not picked up by the scoring metric.)

You may want to use Aragorn or Alice as your starting point.
Then see if you can find tricks that will get a more awesome score for Awsom.
How you choose to do that is up to you, but some ideas are below.

(Reminder: **Don't change evaluation.**  Just build a better argubot.)

In [95]:
awsom_eval = None
with LoggingContext("eval", "INFO"):
    awsom_eval = evaluate.eval_on_characters(argubots.awsom)
rich.print(awsom_eval)

100%|██████████| 10/10 [04:51<00:00, 29.11s/it]


You just spent $0.02 of NLP money to evaluate <LLMAgent Awsom>                                      ]8;id=355577;file:///Users/rayna/Documents/jhu/nlp/NLP-LLM/evaluate.py\evaluate.py]8;;\:]8;id=163744;file:///Users/rayna/Documents/jhu/nlp/NLP-LLM/evaluate.py#296\296]8;;\

<Eval of 10 dialogues: {'engaged': 3.5, 'informed': 3.3, 'intelligent': 3.5, 'moral': 3.3, 'skilled': 7.1, 'TOTAL':
20.7}>
Standard deviations: {'engaged': 1.0801234497346435, 'informed': 0.4830458915396473, 'intelligent': 
0.5270462766947299, 'moral': 0.4830458915396473, 'skilled': 0.8755950357709117, 'TOTAL': 3.0930028559098854}

Comments from participant_overview question:
(Bob) Awsom didn't necessarily disagree with me but rather raised important points about cultural practices, 
individual health needs, and economic factors that influence dietary choices. The conversation was constructive, as
it allowed for a discussion on the complexities surrounding vegetarianism beyond just personal choice.

Awsom could have done better by acknowledging the potential for change and solutions more explicitly, rather than 
focusing solely on the barriers. This would have created a more balanced dialogue about how to promote 
vegetarianism while considering those challenges.
(Bob) Awsom seemed to disagree with my perspective on the flexibility of cultural traditions that include meat and 
the idea that they can be adapted to fit a vegetarian lifestyle. He raised valid points about the deeper meanings 
behind certain cultural practices and how they might not easily lend themselves to replacement with plant-based 
alternatives.

In my opinion, the conversation was constructive. We both engaged in a respectful dialogue, exploring the 
complexities of dietary choices and cultural significance. I appreciated Awsom's insights, and I aimed to convey my
belief in the possibility of adapting traditions.

Awsom could have done better by acknowledging the potential for creativity in modifying traditions more openly. 
While he raised important concerns, a more collaborative approach in exploring solutions could have enriched the 
conversation further.
(Cara) Awsom seemed to disagree with my stance on eating meat, particularly regarding the ethical implications and 
broader considerations like animal rights and environmental impact. In my opinion, the conversation was 
straightforward, with me firmly expressing my views while Awsom tried to challenge them. 

Awsom could have done better by acknowledging my perspective more and perhaps asking more open-ended questions that
invite discussion rather than just presenting counterarguments. This might have led to a more engaging dialogue.
(Cara) Awsom disagreed with me about the ethics of meat consumption and the potential benefits of incorporating 
plant-based foods into my diet. In my opinion, the conversation felt a bit one-sided, as I was clear about my 
preferences, but Awsom kept pushing the idea of trying plant-based options. Awsom could have done better by 
respecting my choices and perhaps asking more about my experiences with meat instead of insisting on alternatives.
(Darius) Awsom didn't outright disagree with me but rather posed thoughtful questions that encouraged a broader 
discussion about public health strategies beyond just vaccination. The conversation was constructive, as it allowed
for an exchange of ideas regarding the importance of vaccination while acknowledging the value of complementary 
measures and historical context.

Awsom could have done better by providing specific examples or data to support his points about alternative public 
health measures. This would have strengthened his arguments and made the discussion more fact-based, which is 
essential in public health discourse. Additionally, he could have asked more direct questions about the limitations
of vaccination alone, which would have further enriched the conversation.
(Darius) Awsom disagreed with my stance on mandatory COVID vaccinations, emphasizing the importance of personal 
autonomy and individual choice in public health. The conversation was a constructive exchange of ideas, but it was 
clear that Awsom was advocating for a more balanced approach that respects personal freedoms while still aiming for
community saf

## [Possible strategy] Prompt engineering

A good first thing to do is to experiment with Alice's prompt.  
The wording and level of detail in the prompt can be quite important.
Often, NLP engineers will change their prompt to try to address 
problems that they've seen in the responses.

Because it's "just" text editing, this won't get full credit by itself unless you make a real discovery.
But it requires intelligence, care, experimentation, and alertness to the language of the responses and the
language of the prompts.  And you'll develop some intuitions about what helps and what doesn't.
It is certainly worthwhile.

Of course, people have tried to develop methods to search for good prompts automatically, or semi-automatically with human guidance.  
So you could additionally try out SAMMO or DSPy -- both have multiple tutorials and are downloadable from github.

If you try this, what worked well for you?

## [Possible strategy] Chain of thought / Planning

The evaluation functions in `evaluate.py` asked each `EvaluationAgent` a "warmup question" before continuing with the real question.  That is an example of chain-of-thought (CoT) reasoning, where the LLM is encouraged to talk through the problem for a few sentences before giving the answer.  CoT sometimes improves performance.

Instead of using one prompt, could you help an `LLMAgent` argubot (like Alice) do better by having think aloud before it gives an answer?  For example, each time the human speaks, your argubot (Awsom) could prompt the LLM to think about the human's ideas/motivations/personality, and to come up with a plan for how to open the human's mind. 

For example, you might structure this as a `Dialogue` among three participants, like this:
> Awsom (to Eve): Do you think COVID vaccines should be mandatory?
>
> Eve: Have you ever gotten vaccinated yourself?<br>
>
> Awsom (private thought): I don't know Eve's opinions yet, so I can't push back.  Eve might be avoiding my question because she doesn't want to get into a political argument.  So let's see if we can get her to express an opinion on something less political.  Maybe something more personal ... like whether vaccines are scary.
>
> Awsom (to Eve): In fact I have, and so have millions of others. But some people seem scared about getting the vaccine.  

One way to trigger this kind of analysis is to present a `Dialogue.script()` to Awsom (or to an observer), and ask an open-ended question about it.  Or you could ask a series of more specific questions.  That is basically what `eval_by_participant` and `eval_by_observer` do.  But here the argubot itself is doing it, rather than the evaluation framework.

Eve would be shown only the turns that are spoken aloud.  However, when analyzing and responding, Awsom would get to see Awsom's own private thoughts as well.


## [Possible strategy] Dense embeddings

BM25 uses sparse embeddings — a document's embedding vector is mostly zeroes, since the non-zero coordinates correspond to the specific words (tokens) that appear in the document.

But perhaps dense embeddings of documents would improve Aragorn by reading the text and abstracting away from the words, in a way that actually cares about word order.  So, try it!

How?  As mentioned earlier in this notebook, you could compute the embeddings yourself and put them in a FAISS index. Or you could figure out how to use OpenAI's [knowledge retrieval](https://platform.openai.com/docs/assistants/tools/knowledge-retrieval) API.

## [Possible strategy] Few-shot prompting

 In this homework, often an agent prompted a language model only with instructions.  Can you find a place where giving a few _examples_ would also improve performance?  You will have to write the examples, and you will have to add them to the sequence of messages that your agent sends to the OpenAI API.  See the sentence-reversal illustration earlier in this notebook.

One good opportunity is in the query formation step of RAG.  This is a tricky task.  The LLM is supposed to state the user's implicit claim in a form that looks like a Kialo claim (or, more precisely, a form that will work well as a Kialo query).  It probably doesn't know what Kialo claims look like.  So you could show it by way of example.  This would also show it what you mean by the user's "implicit claim."


## [Possible strategy] Using tools in the approved way

Aragorn's step 1 (query formation) is basically getting the LLM to generate a function call like
```
kialo_thoughts("A vaccine that was developed very quickly ...")
```
which Aragorn will execute at step 2 (retrieval), sending the results back to the LLM as part of step 3.

In this context, `kialo_thoughts` is an example of a **tool** (that is, a function) that the
LLM can or must use before it gives its response.

The tool is _not_ something that runs on the LLM server.  It is written by you
in Python and executed by you.  The function call above, including the text `"A
vaccine that was ..."`, is the part that is generated by the LLM.

The OpenAI API has [special support](https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models) for calling the LLM in a way that will _allow_ it to generate a tool call ([tools](https://platform.openai.com/docs/api-reference/chat/create#chat-create-tools)) or _force_ it to do so ([tool_choice](https://platform.openai.com/docs/api-reference/chat/create#chat-create-tool_choice)).  You can then send the tool's result back to the LLM [as part of your message sequence](https://platform.openai.com/docs/api-reference/chat/create#chat-create-messages).

So, you could modify Aragorn to use tools properly.  Maybe that will help, simply because the LLM was trained on message sequences that included tool use.  It should know to pay attention to the tool portions of the prompt when they are relevant, and ignore them when they are not.

The `client.chat.completions.create()` method would need to be told about the tool by using the `tools` keyword argument, with a value something the one below.

If `d` is a `Dialogue`, you should be able to call `d.response()` with the `tools` keyword argument.  This will be passed on to `client.chat.completions.create()` as desired.

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "kialo_thoughts",
            "description": "Given a claim by the user, find a similar claim on the Kialo website and return its pro and con responses",
            "parameters": {
                "type": "object",
                "properties": {
                    "search_topic": {
                        "type": "string",
                        "description": "A claim that was made explicitly or implicitly by the user.",
                    },
                },
                "required": ["search_topic"],
            },
        }
    }]

## [Possible strategy] Parallel generation

The chat completions interface allows you to sample $n$ continuations of the prompt in parallel, as we saw with "the apples, bananas, cherries ..." example.  This is efficient because it requires only 1 request to the LLM server and not $n$.  The latency does not scale with $n$.  Nor does the input token cost, since the prompt only has to be encoded once.

Perhaps you can find a way to make use of this?  For example, the query formulation step of RAG could generate $n$ implicit claims instead of just one.  We could then look for claims in the Kialo database that are close to _any_ of those implicit claims.

Another thing to do with multiple completions is to select among them or combine them.  For example, suppose we prompt the LLM to generate completions of the form $(s,t,r)$ where $s$ is an answer, $t$ evaluates that answer, and $r$ is a numerical score or reward based on that evaluation.  ("Write a poem, then tell us about its rhyme and rhythm problems, then give your score.")  
* If we sample multiple completions $(s_1,t_1,r_1), \ldots, (s_n,t_n,r_n)$ in parallel, then we can return the $s_i$ whose $r_i$ is largest.  
* Or if we sample $s$ and then multiple continuations $(t_1,r_1), \ldots, (t_n,r_n)$, then we can return the mean score $\sum_i r_i/n$ as a reduced-variance score for $s$, which averages over diverse textual evaluations that might consider different aspects of $s$.

Note that when you call the chat completions interface with $n > 1$, you specfy just 1 input prompt and get $n$ different output completions.  Since the input prompt must be the same for all outputs, it is necessary to sample all of $(s,t,r)$ or all of $(t,r)$ with a single call to the LLM.

Alternatively, it is possible to reduce latency by submitting multiple requests to the server in parallel (see "async usage" [here](https://pypi.org/project/openai/)).  In this case the input prompts can be different, although you now have to pay to encode all of them separately.  This facility could speed up evaluation without changing its results; that's a worthwhile thing to try for extra credit!


# [Extra credit] Adversarial testing (Anansi)

![image](handinec.png)
Finally, let's test whether our eval metric `evaluate.eval_on_characters` is vulnerable to adversarial gaming.  Remember [Goodhart's Law](https://en.wikipedia.org/wiki/Goodhart%27s_law) ...

Add one more argubot to `argubots.py`.
Call it [Anansi](https://www.britannica.com/topic/Ananse), after the trickster character from folklore.

Can you make Anansi *fool* the judges into giving him a high score?  (Higher than some of the earlier argubots, while actually being worse at the task?)  **Any sneaky way of constructing Anansi's responses is fair game.**  The goal is to do well under automated evaluation on a held-out test set.  That is, Anansi should continue to score highly when talking to a character who is not in `evaluate.dev_chars` = {Bob, Cara, Darius, Eve, TrollFace}, when judged both by the character he is talking to and by Judge Wise.

To do well at this, figure out what the judges "want" -- what they might reward or respond positively to -- and how to give it to them.  This might be done by pure prompt engineering, or with additional computation (perhaps making use of additional LLM calls or other resources).  Again, explain what you did, and discuss how it worked out on the dev set.  Feel free to mention other ideas you had, too.